# 추천

In [40]:
from pyvis.network import Network
import pandas as pd
import re
from collections import Counter
from konlpy.tag import Okt

# 형태소 분석기 초기화
okt = Okt()

# 한글 폰트 설정
from matplotlib import font_manager, rc
font_path = "C:/Windows/Fonts/malgun.ttf"  # 윈도우의 경우
font_name = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font_name)

# 불용어 리스트 정의 (한국어 불용어)
with open(r"\Users\kyn03\OneDrive\바탕 화면\project_file\stopwords-ko(한국어불용어).txt", encoding='utf-8') as f:
    stopwords = f.read().splitlines()

# 추가 불용어 정의
stopwords += ['좀', '것', '달바', '미스트', '스프레이', '1으로', '1이라', '1에', '너무', '느낌', '잘', '그냥', '많이', 
              '있어서', '같아요', '스프레이가', '미스트가', '미스트는', '약간의', '생각보다', '있으며', '살짝', '같아서', 
              '느낌이', '저는', '진짜', '엄청', '합니다', '더', '안', '느낌은', '효과는', '조금', '분들은', '크림미스트라서',
              '크림미스트라', '크림', '크림이', '미스트라', '미스트라서', '그런지', '못', '이거', '완전', '아벤느', '다', '개', '걸']

# 데이터 로드
data = pd.read_excel(r"\Users\kyn03\OneDrive\바탕 화면\project_file\sorted_merged_dalba.xlsx")
data = data.where(pd.notna(data), None)  # 중복된 행은 유지하되, 결측치를 NA로 채움
data = data.reset_index()

# 인덱스 넘버가 3 + 7 * n인 경우 필터링하여 'Aspect2 Mapped Labels' 행 필터링
filtered_data = data.iloc[[i for i in range(len(data)) if (i - 2) % 7 == 0]]

# 새 속성 키워드 "추천" 설정
aspect_keyword = "추천"

# 텍스트 전처리 함수: 한글, 숫자, '+'만 남기고 특수문자 제거 및 불용어 제거
def preprocess_text(text_list):
    cleaned_text = ' '.join([re.sub(r"[^가-힣0-9\+\s]", "", text) for text in text_list])
    cleaned_text = ' '.join([word for word in cleaned_text.split() if word not in stopwords])
    return cleaned_text

# 조사에 따른 방향 색상 및 거리 설정
direction_settings = {
    "타인을 향한 방향": {"color": "lightgreen", "distance": 300},
    "주체를 나타냄": {"color": "skyblue", "distance": 150},
    "대상이 되는 목적어": {"color": "salmon", "distance": 200},
    "기타": {"color": "gray", "distance": 400}  # 방향성이 없는 경우 기본 색상 및 거리
}

# 조사에 따른 방향 파악 함수
def determine_direction(token):
    # 타인을 향한 방향: '-에게', '-께', '-한테' 등의 변형 포함
    if re.search(r"(에게|에게는|에게도|에게서|에게까지|께|께는|께서|께만|한테|한테는|한테서|한테까지|을 위한|를 위한|위하여)$", token):
        return "타인을 향한 방향"
    # 주체를 나타냄: '-가/이', '-은/는', '-에서', '-에' 등의 변형 포함
    elif re.search(r"(가|이|이가|은|는|에는|는지|는가|께서|께서만|에서|에)$", token):
        return "주체를 나타냄"
    # 대상을 나타내는 목적어: '-을/를', '-로', '-으로' 등의 변형 포함
    elif re.search(r"(을|를|를 통해|을 통해서|을 받아|로|으로)$", token):
        return "대상이 되는 목적어"
    # 기타 방향이 없는 경우
    else:
        return "기타"

# 네트워크 그래프 생성을 위한 데이터 수집
nodes_data = []
all_previous_tokens = []

# "추천" 단어가 포함된 셀과 바로 앞 열의 단어 연결
for idx, row in filtered_data.iterrows():
    original_idx = data[data['index'] == row['index']].index[0]
    tokenized_text_index = original_idx + 3  # tokenized_text 행 인덱스
    
    if tokenized_text_index < len(data):
        for col_index in range(2, len(data.columns)):  # 첫 두 열 제외, 각 열을 순회
            tokenized_text = str(data.iat[tokenized_text_index, col_index])
            
            # "추천" 단어 포함 여부 확인
            if "추천" in tokenized_text:
                # 바로 앞 열에 있는 단어 추출
                previous_text = str(data.iat[tokenized_text_index, col_index - 1])
                if pd.notna(previous_text):
                    # 텍스트를 전처리한 후 토큰으로 분리하여 조사 확인 및 방향 설정
                    cleaned_text = preprocess_text([previous_text])
                    tokens = cleaned_text.split()
                    for token in tokens:
                        direction = determine_direction(token)
                        color = direction_settings[direction]["color"]
                        distance = direction_settings[direction]["distance"]
                        nodes_data.append((aspect_keyword, token, color, distance))
                        all_previous_tokens.append(token)

# 상위 10개 단어 추출
counter = Counter(all_previous_tokens)
most_common_tokens = counter.most_common(10)

# 상위 10개 단어 출력
print("상위 10개 자주 등장하는 방향 관련 토큰:")
for word, freq in most_common_tokens:
    print(f"{word}: {freq}")

# 네트워크 그래프 생성
net = Network(notebook=True, cdn_resources="in_line")  # 'in_line' 설정으로 Jupyter Notebook 내에서 표시
net.toggle_physics(True)  # 물리 엔진 활성화로 층위 효과 강조

# "추천" 단어와 그 앞에 나오는 방향 관련 토큰들을 연관 노드로 연결
for main_token, previous_token, color, distance in nodes_data:
    # 메인 토큰 노드 추가
    net.add_node(main_token, label=main_token, color="white", size=20, level=0)
    
    # 연관 노드 추가 (노드 내부 색상 흰색)
    net.add_node(previous_token, label=previous_token, color=color, borderWidth=2, shape="dot", size=15, 
                 font={"color": "black"}, title=previous_token)
    net.add_edge(main_token, previous_token, color=color, length=distance)

# Show interactive network
net.show("interactive_network_recommendation_1.html")


상위 10개 자주 등장하는 방향 관련 토큰:
적극: 10
친구: 9
분들에게: 8
좋아서: 7
건성분들에게: 5
만족합니다: 5
분들께: 4
구매: 4
좋아요: 4
친구가: 4
interactive_network_recommendation_1.html


In [3]:
from pyvis.network import Network
import pandas as pd
import re
from collections import Counter
from konlpy.tag import Okt

# 형태소 분석기 초기화
okt = Okt()

# 한글 폰트 설정
from matplotlib import font_manager, rc
font_path = "C:/Windows/Fonts/malgun.ttf"  # 윈도우의 경우
font_name = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font_name)

# 불용어 리스트 정의 (한국어 불용어)
with open(r"\Users\kyn03\OneDrive\바탕 화면\project_file\stopwords-ko(한국어불용어).txt", encoding='utf-8') as f:
    stopwords = f.read().splitlines()

# 추가 불용어 정의
stopwords += ['좀', '것', '달바', '미스트', '스프레이', '1으로', '1이라', '1에', '너무', '느낌', '잘', '그냥', '많이', 
              '있어서', '같아요', '스프레이가', '미스트가', '미스트는', '약간의', '생각보다', '있으며', '살짝', '같아서', 
              '느낌이', '저는', '진짜', '엄청', '합니다', '더', '안', '느낌은', '효과는', '조금', '분들은', '크림미스트라서',
              '크림미스트라', '크림', '크림이', '미스트라', '미스트라서', '그런지', '못', '이거', '완전', '아벤느', '다', '개', '걸']

# 데이터 로드
data = pd.read_excel(r"\Users\kyn03\OneDrive\바탕 화면\project_file\sorted_merged_dalba.xlsx")
data = data.where(pd.notna(data), None)  # 중복된 행은 유지하되, 결측치를 NA로 채움
data = data.reset_index()

# 인덱스 넘버가 3 + 7 * n인 경우 필터링하여 'Aspect2 Mapped Labels' 행 필터링
filtered_data = data.iloc[[i for i in range(len(data)) if (i - 2) % 7 == 0]]

# 새 속성 키워드 "추천" 설정
aspect_keyword = "추천"

# 텍스트 전처리 함수: 한글, 숫자, '+'만 남기고 특수문자 제거 및 불용어 제거
def preprocess_text(text_list):
    cleaned_text = ' '.join([re.sub(r"[^가-힣0-9\+\s]", "", text) for text in text_list])
    cleaned_text = ' '.join([word for word in cleaned_text.split() if word not in stopwords])
    return cleaned_text

# 조사에 따른 방향 색상 및 거리 설정
direction_settings = {
    "타인을 향한 방향": {"color": "lightgreen", "distance": 300},
    "주체를 나타냄": {"color": "skyblue", "distance": 150},
    "대상이 되는 목적어": {"color": "salmon", "distance": 200},
    "기타": {"color": "gray", "distance": 400}  # 방향성이 없는 경우 기본 색상 및 거리
}

# 조사에 따른 방향 파악 함수
def determine_direction(token):
    # 타인을 향한 방향: '-에게', '-께', '-한테' 등의 변형 포함
    if re.search(r"(에게|에게는|에게도|에게서|에게까지|께|께는|께서|께만|한테|한테는|한테서|한테까지|을 위한|를 위한|위하여)$", token):
        return "타인을 향한 방향"
    # 주체를 나타냄: '-가/이', '-은/는', '-에서', '-에' 등의 변형 포함
    elif re.search(r"(가|이|이가|은|는|에는|는지|는가|께서|께서만|에서|에)$", token):
        return "주체를 나타냄"
    # 대상을 나타내는 목적어: '-을/를', '-로', '-으로' 등의 변형 포함
    elif re.search(r"(을|를|를 통해|을 통해서|을 받아|로|으로)$", token):
        return "대상이 되는 목적어"
    # 기타 방향이 없는 경우
    else:
        return "기타"

# 네트워크 그래프 생성을 위한 데이터 수집 (빈도 포함)
nodes_data = []
all_previous_tokens = []

# "추천" 단어가 포함된 셀과 바로 앞 열의 단어 연결
for idx, row in filtered_data.iterrows():
    original_idx = data[data['index'] == row['index']].index[0]
    tokenized_text_index = original_idx + 3  # tokenized_text 행 인덱스
    
    if tokenized_text_index < len(data):
        for col_index in range(2, len(data.columns)):  # 첫 두 열 제외, 각 열을 순회
            tokenized_text = str(data.iat[tokenized_text_index, col_index])
            
            # "추천" 단어 포함 여부 확인
            if "추천" in tokenized_text:
                # 바로 앞 열에 있는 단어 추출
                previous_text = str(data.iat[tokenized_text_index, col_index - 1])
                if pd.notna(previous_text):
                    # 텍스트를 전처리한 후 토큰으로 분리하여 조사 확인 및 방향 설정
                    cleaned_text = preprocess_text([previous_text])
                    tokens = cleaned_text.split()
                    for token in tokens:
                        direction = determine_direction(token)
                        color = direction_settings[direction]["color"]
                        distance = direction_settings[direction]["distance"]
                        nodes_data.append((aspect_keyword, token, color, distance))
                        all_previous_tokens.append(token)

# 빈도수 카운터 생성
counter = Counter(all_previous_tokens)
node_frequency = dict(counter)  # 각 단어와 그 빈도를 저장하는 딕셔너리

# 상위 10개 단어 출력
print("상위 10개 자주 등장하는 방향 관련 토큰:")
for word, freq in counter.most_common(10):
    print(f"{word}: {freq}")

# 네트워크 그래프 생성
net = Network(notebook=True, cdn_resources="in_line")  # 'in_line' 설정으로 Jupyter Notebook 내에서 표시
net.toggle_physics(True)  # 물리 엔진 활성화로 층위 효과 강조

# "추천" 단어와 그 앞에 나오는 방향 관련 토큰들을 연관 노드로 연결
for main_token, previous_token, color, distance in nodes_data:
    # 메인 토큰 노드 추가
    net.add_node(main_token, label=main_token, color="white", size=20, level=0)
    
    # 연관 노드 추가 (노드 크기 및 툴팁에 빈도수 표시)
    node_size = 15 + node_frequency.get(previous_token, 0) * 2  # 빈도수에 따라 크기 설정
    node_title = f"{previous_token} (빈도: {node_frequency.get(previous_token, 0)})"  # 빈도수 툴팁
    
    net.add_node(previous_token, label=previous_token, color=color, borderWidth=2, shape="dot", size=node_size, 
                 font={"color": "black"}, title=node_title)
    net.add_edge(main_token, previous_token, color=color, length=distance)

# Show interactive network
net.show("interactive_network_recommendation.html")


상위 10개 자주 등장하는 방향 관련 토큰:
적극: 10
친구: 9
분들에게: 8
좋아서: 7
건성분들에게: 5
만족합니다: 5
분들께: 4
구매: 4
좋아요: 4
친구가: 4
interactive_network_recommendation.html


# 냄새

## 긍부정

In [78]:
from pyvis.network import Network
import pandas as pd
import re
from collections import Counter
from konlpy.tag import Okt

# 형태소 분석기 초기화
okt = Okt()

# 불용어 리스트 정의 (한국어 불용어)
with open(r"\Users\kyn03\OneDrive\바탕 화면\project_file\stopwords-ko(한국어불용어).txt", encoding='utf-8') as f:
    stopwords = f.read().splitlines()

# 추가 불용어 정의
stopwords += ['좀', '것', '달바', '미스트', '스프레이', '1으로', '1이라', '1에', '너무', '느낌', '잘', '그냥', '많이', 
              '있어서', '같아요', '스프레이가', '미스트가', '미스트는', '약간의', '생각보다', '있으며', '살짝', '같아서', 
              '느낌이', '저는', '진짜', '엄청', '합니다', '더', '안', '느낌은', '효과는', '조금', '분들은', '크림미스트라서',
              '크림미스트라', '크림', '크림이', '미스트라', '미스트라서', '그런지', '못', '이거', '완전', '아벤느', '다', '개', '걸',
              '매우','음','확','전','제','달바는','막','넘','너무너무','특히','근데','취향이','정말','덜','뭔가',
              '저의','굉장히','아니지만','아니고','해야할까요','제품','시',
              '있는데','샀는데','아니라','기분','썼을것같은데','바꿔봤는데','그런','사봤는데','해야할까요','저에게는',
              '하는데','아주','알겠더라구용','되게','나요','나서','나면서','진심','나니까','저한텐','저한테는','구매했어요',]

# 데이터 로드
data = pd.read_excel(r"\Users\kyn03\OneDrive\바탕 화면\project_file\sorted_merged_dalba.xlsx")
data = data.where(pd.notna(data), None)  # 결측치 처리
data = data.reset_index()

# 인덱스 넘버가 3 + 7 * n인 경우 필터링하여 'Aspect2 Mapped Labels' 행 필터링
filtered_data = data.iloc[[i for i in range(len(data)) if (i - 2) % 7 == 0]]

# 키워드 설정
aspect_keywords = ["향", "향기", "냄새"]

# 텍스트 전처리 함수
def preprocess_text(text):
    """텍스트에서 특수문자 제거 및 불용어 필터링"""
    cleaned_text = re.sub(r"[^가-힣0-9\s]", "", text)
    tokens = [word for word in cleaned_text.split() if word not in stopwords]
    return tokens

# 네트워크 그래프 생성을 위한 데이터 수집
nodes_data = set()  # 중복 방지를 위해 set 사용
all_previous_tokens = []

# 키워드와 관련된 단어 연결
for idx, row in filtered_data.iterrows():
    original_idx = data[data['index'] == row['index']].index[0]
    tokenized_text_index = original_idx + 3  # 'Tokenized Text' 행
    sentiment_row_index = tokenized_text_index - 5  # 감정 데이터는 동일 열의 -5행에 위치

    if tokenized_text_index < len(data):

        for col_index in range(2, len(data.columns)):  # 각 열을 순회
            tokenized_text = str(data.iat[tokenized_text_index, col_index])

            # 감정 데이터 가져오기
            try:
                sentiment_value = str(data.iat[sentiment_row_index, col_index]).strip()
            except Exception as e:
                print(f"[ERROR] 감정 데이터를 가져오는 중 에러 발생: {e}")
                sentiment_value = None

            # Null 또는 빈 문자열 확인
            if not tokenized_text or tokenized_text.strip() == "":
                continue

            # 키워드 포함 여부 확인
            if not any(keyword in tokenized_text for keyword in aspect_keywords):
                continue

            # 키워드가 포함된 경우 처리
            print(f"[INFO] 키워드 포함 텍스트: {tokenized_text}, 감정: {sentiment_value}")

            # 기준 셀의 왼쪽과 오른쪽 셀 값을 추출
            left_text = (
                str(data.iat[tokenized_text_index, col_index - 1])
                if col_index > 2  # 왼쪽 셀이 존재하는 경우
                else None
            )
            right_text = (
                str(data.iat[tokenized_text_index, col_index + 1])
                if col_index < len(data.columns) - 1  # 오른쪽 셀이 존재하는 경우
                else None
            )

            # 제외할 접미사를 리스트로 관리
            EXCLUDED_SUFFIXES = ("고", "도","데", )
            EXCLUDED_KEYWORDS = ["뿌", "사용", "써", "구매","듯","나네","처음","있","나고","분사","촉촉",]
            # 왼쪽 셀 추가
            if left_text and left_text.strip():
                stripped_left_text = left_text.strip()  # 공백 제거
                # 접미사 검사
                if any(stripped_left_text.endswith(suffix) for suffix in EXCLUDED_SUFFIXES):
                    print(f"[INFO] 왼쪽 셀이 '{EXCLUDED_SUFFIXES}' 중 하나로 끝나서 제외됨: {stripped_left_text}")

                elif any(keyword in stripped_left_text for keyword in EXCLUDED_KEYWORDS):
                    print(f"[INFO] 왼쪽 셀이 제외 키워드를 포함하여 제외됨: {stripped_left_text}")
                else:
                    nodes_data.add((tokenized_text, stripped_left_text, tokenized_text_index, col_index))  # set에 추가
                    all_previous_tokens.append(stripped_left_text)

            # 오른쪽 셀 추가
            if right_text and right_text.strip():
                stripped_right_text = right_text.strip()  # 공백 제거
                # 키워드 포함 여부 확인
                if any(keyword in stripped_right_text for keyword in EXCLUDED_KEYWORDS):
                    print(f"[INFO] 오른쪽 셀이 제외 키워드를 포함하여 제외됨: {stripped_right_text}")
                else:
                    nodes_data.add((tokenized_text, right_text.strip(),tokenized_text_index,col_index))  # set에 추가
                    all_previous_tokens.append(right_text.strip())

 

# 빈도수 카운터 생성
if not all_previous_tokens:
    print("[ERROR] 관련된 서술어가 없습니다. 전처리 또는 데이터 로드 로직을 확인하세요.")
else:
    # 감정별로 빈도수를 따로 저장하기 위한 딕셔너리
    sentiment_frequency = {
        "긍정": Counter(),
        "부정": Counter(),
        "중립": Counter()
    }

    # 전체 빈도수 저장
    total_frequency = Counter()

    # 빈도수 계산
    for tokenized_text, related_text, tokenized_text_index, col_index in nodes_data:
        # 감정 값 가져오기
        try:
            sentiment_value = str(data.iat[tokenized_text_index - 5, col_index]).strip()
            # B-긍정, I-긍정 같은 값을 "긍정"으로 통합
            if sentiment_value.startswith("B-") or sentiment_value.startswith("I-"):
                sentiment_value = sentiment_value[2:]
        except Exception as e:
            print(f"[ERROR] 감정 데이터를 가져오는 중 에러 발생: {e}")
            sentiment_value = None

        # 유효한 감정 값인지 확인
        if sentiment_value in sentiment_frequency:
            sentiment_frequency[sentiment_value][related_text] += 1
            total_frequency[related_text] += 1  # 전체 빈도수 업데이트




[INFO] 키워드 포함 텍스트: 향료, 감정: O
[INFO] 키워드 포함 텍스트: 향이, 감정: O
[INFO] 키워드 포함 텍스트: 향과, 감정: B-부정
[INFO] 키워드 포함 텍스트: 향료, 감정: O
[INFO] 키워드 포함 텍스트: 향이, 감정: O
[INFO] 키워드 포함 텍스트: 향과, 감정: B-부정
[INFO] 키워드 포함 텍스트: 향료, 감정: O
[INFO] 키워드 포함 텍스트: 향이, 감정: O
[INFO] 키워드 포함 텍스트: 향과, 감정: B-부정
[INFO] 키워드 포함 텍스트: 향이, 감정: I-부정
[INFO] 키워드 포함 텍스트: 영향으로부터, 감정: I-긍정
[INFO] 키워드 포함 텍스트: 향료, 감정: O
[INFO] 키워드 포함 텍스트: 향이, 감정: O
[INFO] 키워드 포함 텍스트: 향과, 감정: O
[INFO] 키워드 포함 텍스트: 향이, 감정: B-부정
[INFO] 키워드 포함 텍스트: 향이, 감정: B-부정
[INFO] 키워드 포함 텍스트: 향료, 감정: O
[INFO] 키워드 포함 텍스트: 향이, 감정: O
[INFO] 키워드 포함 텍스트: 향과, 감정: O
[INFO] 키워드 포함 텍스트: 향료, 감정: O
[INFO] 키워드 포함 텍스트: 향이, 감정: O
[INFO] 키워드 포함 텍스트: 향과, 감정: B-부정
[INFO] 키워드 포함 텍스트: 향료, 감정: O
[INFO] 키워드 포함 텍스트: 향이, 감정: O
[INFO] 키워드 포함 텍스트: 향과, 감정: B-부정
[INFO] 키워드 포함 텍스트: 향도, 감정: O
[INFO] 오른쪽 셀이 제외 키워드를 포함하여 제외됨: 써보고
[INFO] 키워드 포함 텍스트: 향은, 감정: B-긍정
[INFO] 키워드 포함 텍스트: 향도, 감정: I-긍정
[INFO] 키워드 포함 텍스트: 향이, 감정: B-긍정
[INFO] 키워드 포함 텍스트: 향이라, 감정: I-긍정
[INFO] 키워드 포함 텍스트: 향이, 감정: I-긍정
[INFO] 키워드 포함 텍스트:

In [79]:
# 네트워크 그래프 생성
net = Network(notebook=True, cdn_resources="in_line")
net.toggle_physics(True)

# 중심 노드와 연관 노드 간 연결 정보 기록
connected_nodes = {}
center_frequency = Counter()

# 그래프 노드 및 엣지 추가
EXCLUDED_MAIN_KEYWORDS = ["영향", "상향", "지향", "의향", "취향"]

for main_token, related_token, sentiment_row_index, col_index in nodes_data:
    main_node_key = main_token.strip()  # 중심 노드 키 생성
    print(f"[DEBUG] Processing pair: main_token={main_token}, related_token={related_token}")

    # 유효성 검증 및 불용어 필터링
    if not re.match(r"^[가-힣]+$", main_node_key) or not re.match(r"^[가-힣]+$", related_token):
        print(f"[INFO] main_token 또는 related_token이 유효하지 않음: {main_token}, {related_token}")
        continue
    if main_token in stopwords or related_token in stopwords:
        print(f"[INFO] 불용어로 제외됨: {main_token} 또는 {related_token}")
        continue
    if any(keyword in main_node_key for keyword in EXCLUDED_MAIN_KEYWORDS):
        print(f"[INFO] 중심 노드가 제외 키워드를 포함하여 추가되지 않음: {main_node_key}")
        continue

    # 감정 값 가져오기
    related_token_sentiment = None
    try:
        related_token_sentiment = str(data.iat[sentiment_row_index - 5, col_index]).strip()
        if related_token_sentiment.startswith("B-") or related_token_sentiment.startswith("I-"):
            related_token_sentiment = related_token_sentiment[2:]  # 통합
    except Exception as e:
        print(f"[ERROR] related_token 감정 데이터 가져오기 실패: {e}")
        continue

    # 연관 노드 추가
    related_token_frequency = sentiment_frequency.get(related_token_sentiment, {}).get(related_token, 0)
    if related_token_frequency >= 1:
        if "중립" in related_token_sentiment or "O" in related_token_sentiment:
            print(f"[INFO] 연관 노드가 '중립' 또는 'O' 감정으로 추가되지 않음: {related_token}")
            continue

        related_node_key = f"{related_token}_{related_token_sentiment}"
        if related_node_key not in net.get_nodes():
            node_size = min(15 + related_token_frequency * 2, 50)
            related_token_color = {
                "긍정": "#6EC664",
                "중립": "#BDC3C7",
                "부정": "#E74C3C"
            }.get(related_token_sentiment, "black")
            net.add_node(
                related_node_key,
                label=related_token,
                color=related_token_color,
                size=node_size,
                title=f"{related_token} (빈도: {related_token_frequency})"
            )
            print(f"[DEBUG] 추가된 연관 노드: {related_node_key}")

        # 엣지 및 중심 노드 추가
        if main_node_key not in net.get_nodes():
            net.add_node(
                main_node_key,
                label=main_token,
                color={"background": "white", "border": "black"},
                size=25,
                title=f"{main_token} (빈도: 0)"  # 빈도수는 추후 갱신
            )
        # 엣지 추가
        net.add_edge(main_node_key, related_node_key, color=related_token_color)
        print(f"[DEBUG] 엣지 추가됨: {main_node_key} -> {related_node_key}")
        connected_nodes.setdefault(main_node_key, []).append(related_node_key)

# 중심 노드 빈도 계산 (화면에 표시된 연관 노드 기반)
center_frequency.clear()  # 기존 빈도수 초기화
for main_node_key, related_nodes in connected_nodes.items():
    for related_node_key in related_nodes:
        related_token = related_node_key.split("_")[0]
        related_token_sentiment = related_node_key.split("_")[1]
        related_token_frequency = sentiment_frequency.get(related_token_sentiment, {}).get(related_token, 0)
        center_frequency[main_node_key] += related_token_frequency
        print(f"[DEBUG] 중심 노드 '{main_node_key}'에 '{related_token}'의 빈도 {related_token_frequency} 추가됨")

# 중심 노드 빈도 업데이트 (화면에 표시된 값으로 갱신)
for node in net.nodes:
    if node["id"] in center_frequency:
        node["title"] = f"{node['id']} (빈도: {center_frequency[node['id']]})"
        print(f"[DEBUG] 중심 노드 '{node['id']}'의 빈도 갱신됨: {center_frequency[node['id']]}")

# 디버깅: 최종 중심 노드 빈도수 출력
print("[DEBUG] 최종 중심 노드 빈도수:")
for node, freq in center_frequency.items():
    print(f"{node}: {freq}")

# 네트워크 그래프 시각화
net.show("scent_positive_negative.html")


[DEBUG] Processing pair: main_token=향도, related_token=.
[INFO] main_token 또는 related_token이 유효하지 않음: 향도, .
[DEBUG] Processing pair: main_token=향, related_token=우드같은
[DEBUG] Processing pair: main_token=향, related_token=.
[INFO] main_token 또는 related_token이 유효하지 않음: 향, .
[DEBUG] Processing pair: main_token=향이, related_token=은은한
[DEBUG] 추가된 연관 노드: 은은한_긍정
[DEBUG] 엣지 추가됨: 향이 -> 은은한_긍정
[DEBUG] Processing pair: main_token=조향사와, related_token=콜라보를
[DEBUG] Processing pair: main_token=냄새가, related_token=별로에요
[DEBUG] 추가된 연관 노드: 별로에요_부정
[DEBUG] 엣지 추가됨: 냄새가 -> 별로에요_부정
[DEBUG] Processing pair: main_token=향이였습니다, related_token=.
[INFO] main_token 또는 related_token이 유효하지 않음: 향이였습니다, .
[DEBUG] Processing pair: main_token=ㅜㅜ향, related_token=머리아픔
[INFO] main_token 또는 related_token이 유효하지 않음: ㅜㅜ향, 머리아픔
[DEBUG] Processing pair: main_token=향이, related_token=전
[INFO] 불용어로 제외됨: 향이 또는 전
[DEBUG] Processing pair: main_token=향수같기도, related_token=하면서
[INFO] 불용어로 제외됨: 향수같기도 또는 하면서
[DEBUG] Processing pair: main_token=

In [74]:
import pandas as pd
import re

# 네트워크에 표시된 모든 노드 데이터 가져오기
all_nodes_data = []
for node in net.nodes:
    # 노드 정보 추출
    node_id = node["id"]
    node_label = node.get("label", "")
    node_color = node.get("color", "")
    node_title = node.get("title", "")  # 빈도수 정보 포함 가능

    # 노드 종류 결정
    if "_" in node_id:  # "_"이 포함된 경우 연관 노드
        node_type = "연관 노드"
    else:  # "_"이 없는 경우 중심 노드
        node_type = "중심 노드"

    # 감정 추출 (색상 기반)
    if node_color == "#6EC664":  # 연한 녹색
        node_sentiment = "긍정"
    elif node_color == "#BDC3C7":  # 연한 회색 (중립)
        node_sentiment = "중립"
    elif node_color == "#E74C3C":  # 연한 빨간색 (부정)
        node_sentiment = "부정"
    else:
        node_sentiment = "무감정"

    # 빈도수 추출 (Title에서 추출)
    try:
        node_frequency = int(re.search(r"\(빈도: (\d+)\)", node_title).group(1))
    except AttributeError:
        node_frequency = 0  # 빈도수 정보가 없으면 0으로 설정

    # 데이터 추가
    all_nodes_data.append({
        "Node ID": node_id,
        "Node Type": node_type,
        "Label": node_label,
        "Sentiment": node_sentiment,
        "Frequency": node_frequency,
        "Source Nodes": []  # 초기값으로 빈 리스트 설정 (연관 노드일 경우 추가적으로 설정)
    })

# DataFrame으로 변환
all_nodes_df = pd.DataFrame(all_nodes_data)

# 디버깅: 모든 노드 출력
print("[DEBUG] 모든 노드 데이터:")
print(all_nodes_df.head(10))

# 디버깅: 모든 엣지 데이터 출력
print("[DEBUG] 모든 엣지 데이터:")
for edge in net.edges:
    print(f"Source: {edge['from']}, Target: {edge['to']}")

# 엣지 데이터를 활용하여 Source Nodes 설정
for edge in net.edges:
    source = edge["from"]
    target = edge["to"]

    # 디버깅: 현재 처리 중인 엣지
    print(f"[DEBUG] 처리 중인 엣지: Source={source}, Target={target}")

    # 타겟 노드가 연관 노드인 경우 Source Nodes에 소스 노드 추가
    if "_" in target:  # 타겟 노드가 연관 노드
        target_idx = all_nodes_df[all_nodes_df["Node ID"] == target].index
        if not target_idx.empty:
            current_sources = all_nodes_df.at[target_idx[0], "Source Nodes"]
            if isinstance(current_sources, list):  # 리스트 형태인 경우
                current_sources.append(source)
            else:  # 초기값이 없거나 비어 있을 경우
                current_sources = [source]
            # Source Nodes 갱신
            all_nodes_df.at[target_idx[0], "Source Nodes"] = current_sources

# Source Nodes를 문자열로 변환 (콤마로 연결)
all_nodes_df["Source Nodes"] = all_nodes_df["Source Nodes"].apply(
    lambda x: ", ".join(x) if isinstance(x, list) else ""
)

# 정렬: 중심 노드 기준, 빈도수 기준 내림차순
all_nodes_df = all_nodes_df.sort_values(by=["Node Type", "Frequency"], ascending=[True, False])

# 결과 저장
output_file = "network_nodes_data_scent_positive_negative.xlsx"
all_nodes_df.to_excel(output_file, index=False)

print(f"Processed node data saved to '{output_file}'")

[DEBUG] 모든 노드 데이터:
   Node ID Node Type Label Sentiment  Frequency Source Nodes
0   은은한_긍정     연관 노드   은은한        긍정         19           []
1       향이     중심 노드    향이       무감정       1766           []
2  별로에요_부정     연관 노드  별로에요        부정          1           []
3      냄새가     중심 노드   냄새가       무감정        234           []
4   좋아서_긍정     연관 노드   좋아서        긍정         19           []
5       향도     중심 노드    향도       무감정       8825           []
6   제품의_긍정     연관 노드   제품의        긍정          2           []
7       향을     중심 노드    향을       무감정         18           []
8   강하지_긍정     연관 노드   강하지        긍정          7           []
9    좋고_긍정     연관 노드    좋고        긍정        100           []
[DEBUG] 모든 엣지 데이터:
Source: 향이, Target: 은은한_긍정
Source: 냄새가, Target: 별로에요_부정
Source: 향도, Target: 좋아서_긍정
Source: 향을, Target: 제품의_긍정
Source: 향도, Target: 강하지_긍정
Source: 향도, Target: 좋고_긍정
Source: 향이, Target: 장점은_긍정
Source: 향이, Target: 호불호가_긍정
Source: 향, Target: 흡수_긍정
Source: 향도, Target: 괜찮고_긍정
Source: 향은, Target: 은

## 긍정만

In [69]:
# 네트워크 그래프 생성
net = Network(notebook=True, cdn_resources="in_line")
net.toggle_physics(True)

# 중심 노드와 연관 노드 간 연결 정보 기록
connected_nodes = {}
center_frequency = Counter()

# 그래프 노드 및 엣지 추가
EXCLUDED_MAIN_KEYWORDS = ["영향", "상향", "지향", "의향", "취향"]

for main_token, related_token, sentiment_row_index, col_index in nodes_data:
    main_node_key = main_token.strip()  # 중심 노드 키 생성
    print(f"[DEBUG] Processing pair: main_token={main_token}, related_token={related_token}")

    # 유효성 검증 및 불용어 필터링
    if not re.match(r"^[가-힣]+$", main_node_key) or not re.match(r"^[가-힣]+$", related_token):
        print(f"[INFO] main_token 또는 related_token이 유효하지 않음: {main_token}, {related_token}")
        continue
    if main_token in stopwords or related_token in stopwords:
        print(f"[INFO] 불용어로 제외됨: {main_token} 또는 {related_token}")
        continue
    if any(keyword in main_node_key for keyword in EXCLUDED_MAIN_KEYWORDS):
        print(f"[INFO] 중심 노드가 제외 키워드를 포함하여 추가되지 않음: {main_node_key}")
        continue

    # 감정 값 가져오기
    related_token_sentiment = None
    try:
        related_token_sentiment = str(data.iat[sentiment_row_index - 5, col_index]).strip()
        if related_token_sentiment.startswith("B-") or related_token_sentiment.startswith("I-"):
            related_token_sentiment = related_token_sentiment[2:]  # 통합
    except Exception as e:
        print(f"[ERROR] related_token 감정 데이터 가져오기 실패: {e}")
        continue

    # 연관 노드 추가
    related_token_frequency = sentiment_frequency.get(related_token_sentiment, {}).get(related_token, 0)
    if related_token_frequency >= 1:
        if "중립" in related_token_sentiment or "부정" in related_token_sentiment or "O" in related_token_sentiment:
            print(f"[INFO] 연관 노드가 '중립', 부정', 또는 'O' 감정으로 추가되지 않음: {related_token}")
            continue

        related_node_key = f"{related_token}_{related_token_sentiment}"
        if related_node_key not in net.get_nodes():
            node_size = min(15 + related_token_frequency * 2, 50)
            related_token_color = {
                "긍정": "#6EC664",
                "중립": "#BDC3C7",
                "부정": "#E74C3C"
            }.get(related_token_sentiment, "black")
            net.add_node(
                related_node_key,
                label=related_token,
                color=related_token_color,
                size=node_size,
                title=f"{related_token} (빈도: {related_token_frequency})"
            )
            print(f"[DEBUG] 추가된 연관 노드: {related_node_key}")

        # 엣지 및 중심 노드 추가
        if main_node_key not in net.get_nodes():
            net.add_node(
                main_node_key,
                label=main_token,
                color={"background": "white", "border": "black"},
                size=25,
                title=f"{main_token} (빈도: 0)"  # 빈도수는 추후 갱신
            )
        # 엣지 추가
        net.add_edge(main_node_key, related_node_key, color=related_token_color)
        print(f"[DEBUG] 엣지 추가됨: {main_node_key} -> {related_node_key}")
        connected_nodes.setdefault(main_node_key, []).append(related_node_key)

# 중심 노드 빈도 계산 (화면에 표시된 연관 노드 기반)
center_frequency.clear()  # 기존 빈도수 초기화
for main_node_key, related_nodes in connected_nodes.items():
    for related_node_key in related_nodes:
        related_token = related_node_key.split("_")[0]
        related_token_sentiment = related_node_key.split("_")[1]
        related_token_frequency = sentiment_frequency.get(related_token_sentiment, {}).get(related_token, 0)
        center_frequency[main_node_key] += related_token_frequency
        print(f"[DEBUG] 중심 노드 '{main_node_key}'에 '{related_token}'의 빈도 {related_token_frequency} 추가됨")

# 중심 노드 빈도 업데이트 (화면에 표시된 값으로 갱신)
for node in net.nodes:
    if node["id"] in center_frequency:
        node["title"] = f"{node['id']} (빈도: {center_frequency[node['id']]})"
        print(f"[DEBUG] 중심 노드 '{node['id']}'의 빈도 갱신됨: {center_frequency[node['id']]}")

# 디버깅: 최종 중심 노드 빈도수 출력
print("[DEBUG] 최종 중심 노드 빈도수:")
for node, freq in center_frequency.items():
    print(f"{node}: {freq}")

# 네트워크 그래프 시각화
net.show("scent_positive.html")


[DEBUG] Processing pair: main_token=향도, related_token=.
[INFO] main_token 또는 related_token이 유효하지 않음: 향도, .
[DEBUG] Processing pair: main_token=향, related_token=우드같은
[DEBUG] Processing pair: main_token=향, related_token=.
[INFO] main_token 또는 related_token이 유효하지 않음: 향, .
[DEBUG] Processing pair: main_token=향이, related_token=은은한
[DEBUG] 추가된 연관 노드: 은은한_긍정
[DEBUG] 엣지 추가됨: 향이 -> 은은한_긍정
[DEBUG] Processing pair: main_token=조향사와, related_token=콜라보를
[DEBUG] Processing pair: main_token=냄새가, related_token=별로에요
[INFO] 연관 노드가 '중립', 부정', 또는 'O' 감정으로 추가되지 않음: 별로에요
[DEBUG] Processing pair: main_token=향이였습니다, related_token=.
[INFO] main_token 또는 related_token이 유효하지 않음: 향이였습니다, .
[DEBUG] Processing pair: main_token=ㅜㅜ향, related_token=머리아픔
[INFO] main_token 또는 related_token이 유효하지 않음: ㅜㅜ향, 머리아픔
[DEBUG] Processing pair: main_token=향이, related_token=전
[INFO] 불용어로 제외됨: 향이 또는 전
[DEBUG] Processing pair: main_token=향수같기도, related_token=하면서
[INFO] 불용어로 제외됨: 향수같기도 또는 하면서
[DEBUG] Processing pair: main_token=향도, rel

In [70]:
import pandas as pd
import re

# 네트워크에 표시된 모든 노드 데이터 가져오기
all_nodes_data = []
for node in net.nodes:
    # 노드 정보 추출
    node_id = node["id"]
    node_label = node.get("label", "")
    node_color = node.get("color", "")
    node_title = node.get("title", "")  # 빈도수 정보 포함 가능

    # 노드 종류 결정
    if "_" in node_id:  # "_"이 포함된 경우 연관 노드
        node_type = "연관 노드"
    else:  # "_"이 없는 경우 중심 노드
        node_type = "중심 노드"

    # 감정 추출 (색상 기반)
    if node_color == "#6EC664":  # 연한 녹색
        node_sentiment = "긍정"
    elif node_color == "#BDC3C7":  # 연한 회색 (중립)
        node_sentiment = "중립"
    elif node_color == "#E74C3C":  # 연한 빨간색 (부정)
        node_sentiment = "부정"
    else:
        node_sentiment = "무감정"

    # 빈도수 추출 (Title에서 추출)
    try:
        node_frequency = int(re.search(r"\(빈도: (\d+)\)", node_title).group(1))
    except AttributeError:
        node_frequency = 0  # 빈도수 정보가 없으면 0으로 설정

    # 데이터 추가
    all_nodes_data.append({
        "Node ID": node_id,
        "Node Type": node_type,
        "Label": node_label,
        "Sentiment": node_sentiment,
        "Frequency": node_frequency,
        "Source Nodes": []  # 초기값으로 빈 리스트 설정 (연관 노드일 경우 추가적으로 설정)
    })

# DataFrame으로 변환
all_nodes_df = pd.DataFrame(all_nodes_data)

# 디버깅: 모든 노드 출력
print("[DEBUG] 모든 노드 데이터:")
print(all_nodes_df.head(10))

# 디버깅: 모든 엣지 데이터 출력
print("[DEBUG] 모든 엣지 데이터:")
for edge in net.edges:
    print(f"Source: {edge['from']}, Target: {edge['to']}")

# 엣지 데이터를 활용하여 Source Nodes 설정
for edge in net.edges:
    source = edge["from"]
    target = edge["to"]

    # 디버깅: 현재 처리 중인 엣지
    print(f"[DEBUG] 처리 중인 엣지: Source={source}, Target={target}")

    # 타겟 노드가 연관 노드인 경우 Source Nodes에 소스 노드 추가
    if "_" in target:  # 타겟 노드가 연관 노드
        target_idx = all_nodes_df[all_nodes_df["Node ID"] == target].index
        if not target_idx.empty:
            current_sources = all_nodes_df.at[target_idx[0], "Source Nodes"]
            if isinstance(current_sources, list):  # 리스트 형태인 경우
                current_sources.append(source)
            else:  # 초기값이 없거나 비어 있을 경우
                current_sources = [source]
            # Source Nodes 갱신
            all_nodes_df.at[target_idx[0], "Source Nodes"] = current_sources

# Source Nodes를 문자열로 변환 (콤마로 연결)
all_nodes_df["Source Nodes"] = all_nodes_df["Source Nodes"].apply(
    lambda x: ", ".join(x) if isinstance(x, list) else ""
)

# 정렬: 중심 노드 기준, 빈도수 기준 내림차순
all_nodes_df = all_nodes_df.sort_values(by=["Node Type", "Frequency"], ascending=[True, False])

# 결과 저장
output_file = "network_nodes_data_scent_positive.xlsx"
all_nodes_df.to_excel(output_file, index=False)

print(f"Processed node data saved to '{output_file}'")

[DEBUG] 모든 노드 데이터:
   Node ID Node Type Label Sentiment  Frequency Source Nodes
0   은은한_긍정     연관 노드   은은한        긍정         19           []
1       향이     중심 노드    향이       무감정       1593           []
2   좋아서_긍정     연관 노드   좋아서        긍정         19           []
3       향도     중심 노드    향도       무감정       8817           []
4   제품의_긍정     연관 노드   제품의        긍정          2           []
5       향을     중심 노드    향을       무감정         10           []
6   강하지_긍정     연관 노드   강하지        긍정          7           []
7    좋고_긍정     연관 노드    좋고        긍정        100           []
8   장점은_긍정     연관 노드   장점은        긍정          1           []
9  호불호가_긍정     연관 노드  호불호가        긍정          1           []
[DEBUG] 모든 엣지 데이터:
Source: 향이, Target: 은은한_긍정
Source: 향도, Target: 좋아서_긍정
Source: 향을, Target: 제품의_긍정
Source: 향도, Target: 강하지_긍정
Source: 향도, Target: 좋고_긍정
Source: 향이, Target: 장점은_긍정
Source: 향이, Target: 호불호가_긍정
Source: 향, Target: 흡수_긍정
Source: 향도, Target: 괜찮고_긍정
Source: 향은, Target: 은은한_긍정
Source: 향이, Target: 산뜻한

## 부정만

In [67]:
# 네트워크 그래프 생성
net = Network(notebook=True, cdn_resources="in_line")
net.toggle_physics(True)

# 중심 노드와 연관 노드 간 연결 정보 기록
connected_nodes = {}
center_frequency = Counter()

# 그래프 노드 및 엣지 추가
EXCLUDED_MAIN_KEYWORDS = ["영향", "상향", "지향", "의향", "취향"]

for main_token, related_token, sentiment_row_index, col_index in nodes_data:
    main_node_key = main_token.strip()  # 중심 노드 키 생성
    print(f"[DEBUG] Processing pair: main_token={main_token}, related_token={related_token}")

    # 유효성 검증 및 불용어 필터링
    if not re.match(r"^[가-힣]+$", main_node_key) or not re.match(r"^[가-힣]+$", related_token):
        print(f"[INFO] main_token 또는 related_token이 유효하지 않음: {main_token}, {related_token}")
        continue
    if main_token in stopwords or related_token in stopwords:
        print(f"[INFO] 불용어로 제외됨: {main_token} 또는 {related_token}")
        continue
    if any(keyword in main_node_key for keyword in EXCLUDED_MAIN_KEYWORDS):
        print(f"[INFO] 중심 노드가 제외 키워드를 포함하여 추가되지 않음: {main_node_key}")
        continue

    # 감정 값 가져오기
    related_token_sentiment = None
    try:
        related_token_sentiment = str(data.iat[sentiment_row_index - 5, col_index]).strip()
        if related_token_sentiment.startswith("B-") or related_token_sentiment.startswith("I-"):
            related_token_sentiment = related_token_sentiment[2:]  # 통합
    except Exception as e:
        print(f"[ERROR] related_token 감정 데이터 가져오기 실패: {e}")
        continue

    # 연관 노드 추가
    related_token_frequency = sentiment_frequency.get(related_token_sentiment, {}).get(related_token, 0)
    if related_token_frequency >= 1:
        if "중립" in related_token_sentiment or "긍정" in related_token_sentiment or "O" in related_token_sentiment:
            print(f"[INFO] 연관 노드가 '중립', 긍정', 또는 'O' 감정으로 추가되지 않음: {related_token}")
            continue

        related_node_key = f"{related_token}_{related_token_sentiment}"
        if related_node_key not in net.get_nodes():
            node_size = min(15 + related_token_frequency * 2, 50)
            related_token_color = {
                "긍정": "#6EC664",
                "중립": "#BDC3C7",
                "부정": "#E74C3C"
            }.get(related_token_sentiment, "black")
            net.add_node(
                related_node_key,
                label=related_token,
                color=related_token_color,
                size=node_size,
                title=f"{related_token} (빈도: {related_token_frequency})"
            )
            print(f"[DEBUG] 추가된 연관 노드: {related_node_key}")

        # 엣지 및 중심 노드 추가
        if main_node_key not in net.get_nodes():
            net.add_node(
                main_node_key,
                label=main_token,
                color={"background": "white", "border": "black"},
                size=25,
                title=f"{main_token} (빈도: 0)"  # 빈도수는 추후 갱신
            )
        # 엣지 추가
        net.add_edge(main_node_key, related_node_key, color=related_token_color)
        print(f"[DEBUG] 엣지 추가됨: {main_node_key} -> {related_node_key}")
        connected_nodes.setdefault(main_node_key, []).append(related_node_key)

# 중심 노드 빈도 계산 (화면에 표시된 연관 노드 기반)
center_frequency.clear()  # 기존 빈도수 초기화
for main_node_key, related_nodes in connected_nodes.items():
    for related_node_key in related_nodes:
        related_token = related_node_key.split("_")[0]
        related_token_sentiment = related_node_key.split("_")[1]
        related_token_frequency = sentiment_frequency.get(related_token_sentiment, {}).get(related_token, 0)
        center_frequency[main_node_key] += related_token_frequency
        print(f"[DEBUG] 중심 노드 '{main_node_key}'에 '{related_token}'의 빈도 {related_token_frequency} 추가됨")

# 중심 노드 빈도 업데이트 (화면에 표시된 값으로 갱신)
for node in net.nodes:
    if node["id"] in center_frequency:
        node["title"] = f"{node['id']} (빈도: {center_frequency[node['id']]})"
        print(f"[DEBUG] 중심 노드 '{node['id']}'의 빈도 갱신됨: {center_frequency[node['id']]}")

# 디버깅: 최종 중심 노드 빈도수 출력
print("[DEBUG] 최종 중심 노드 빈도수:")
for node, freq in center_frequency.items():
    print(f"{node}: {freq}")

# 네트워크 그래프 시각화
net.show("scent_negative.html")


[DEBUG] Processing pair: main_token=향도, related_token=.
[INFO] main_token 또는 related_token이 유효하지 않음: 향도, .
[DEBUG] Processing pair: main_token=향, related_token=우드같은
[DEBUG] Processing pair: main_token=향, related_token=.
[INFO] main_token 또는 related_token이 유효하지 않음: 향, .
[DEBUG] Processing pair: main_token=향이, related_token=은은한
[INFO] 연관 노드가 '중립', 긍정', 또는 'O' 감정으로 추가되지 않음: 은은한
[DEBUG] Processing pair: main_token=조향사와, related_token=콜라보를
[DEBUG] Processing pair: main_token=냄새가, related_token=별로에요
[DEBUG] 추가된 연관 노드: 별로에요_부정
[DEBUG] 엣지 추가됨: 냄새가 -> 별로에요_부정
[DEBUG] Processing pair: main_token=향이였습니다, related_token=.
[INFO] main_token 또는 related_token이 유효하지 않음: 향이였습니다, .
[DEBUG] Processing pair: main_token=ㅜㅜ향, related_token=머리아픔
[INFO] main_token 또는 related_token이 유효하지 않음: ㅜㅜ향, 머리아픔
[DEBUG] Processing pair: main_token=향이, related_token=전
[INFO] 불용어로 제외됨: 향이 또는 전
[DEBUG] Processing pair: main_token=향수같기도, related_token=하면서
[INFO] 불용어로 제외됨: 향수같기도 또는 하면서
[DEBUG] Processing pair: main_token=향도, r

In [68]:
import pandas as pd
import re

# 네트워크에 표시된 모든 노드 데이터 가져오기
all_nodes_data = []
for node in net.nodes:
    # 노드 정보 추출
    node_id = node["id"]
    node_label = node.get("label", "")
    node_color = node.get("color", "")
    node_title = node.get("title", "")  # 빈도수 정보 포함 가능

    # 노드 종류 결정
    if "_" in node_id:  # "_"이 포함된 경우 연관 노드
        node_type = "연관 노드"
    else:  # "_"이 없는 경우 중심 노드
        node_type = "중심 노드"

    # 감정 추출 (색상 기반)
    if node_color == "#6EC664":  # 연한 녹색
        node_sentiment = "긍정"
    elif node_color == "#BDC3C7":  # 연한 회색 (중립)
        node_sentiment = "중립"
    elif node_color == "#E74C3C":  # 연한 빨간색 (부정)
        node_sentiment = "부정"
    else:
        node_sentiment = "무감정"

    # 빈도수 추출 (Title에서 추출)
    try:
        node_frequency = int(re.search(r"\(빈도: (\d+)\)", node_title).group(1))
    except AttributeError:
        node_frequency = 0  # 빈도수 정보가 없으면 0으로 설정

    # 데이터 추가
    all_nodes_data.append({
        "Node ID": node_id,
        "Node Type": node_type,
        "Label": node_label,
        "Sentiment": node_sentiment,
        "Frequency": node_frequency,
        "Source Nodes": []  # 초기값으로 빈 리스트 설정 (연관 노드일 경우 추가적으로 설정)
    })

# DataFrame으로 변환
all_nodes_df = pd.DataFrame(all_nodes_data)

# 디버깅: 모든 노드 출력
print("[DEBUG] 모든 노드 데이터:")
print(all_nodes_df.head(10))

# 디버깅: 모든 엣지 데이터 출력
print("[DEBUG] 모든 엣지 데이터:")
for edge in net.edges:
    print(f"Source: {edge['from']}, Target: {edge['to']}")

# 엣지 데이터를 활용하여 Source Nodes 설정
for edge in net.edges:
    source = edge["from"]
    target = edge["to"]

    # 디버깅: 현재 처리 중인 엣지
    print(f"[DEBUG] 처리 중인 엣지: Source={source}, Target={target}")

    # 타겟 노드가 연관 노드인 경우 Source Nodes에 소스 노드 추가
    if "_" in target:  # 타겟 노드가 연관 노드
        target_idx = all_nodes_df[all_nodes_df["Node ID"] == target].index
        if not target_idx.empty:
            current_sources = all_nodes_df.at[target_idx[0], "Source Nodes"]
            if isinstance(current_sources, list):  # 리스트 형태인 경우
                current_sources.append(source)
            else:  # 초기값이 없거나 비어 있을 경우
                current_sources = [source]
            # Source Nodes 갱신
            all_nodes_df.at[target_idx[0], "Source Nodes"] = current_sources

# Source Nodes를 문자열로 변환 (콤마로 연결)
all_nodes_df["Source Nodes"] = all_nodes_df["Source Nodes"].apply(
    lambda x: ", ".join(x) if isinstance(x, list) else ""
)

# 정렬: 중심 노드 기준, 빈도수 기준 내림차순
all_nodes_df = all_nodes_df.sort_values(by=["Node Type", "Frequency"], ascending=[True, False])

# 결과 저장
output_file = "network_nodes_data_scent_negative.xlsx"
all_nodes_df.to_excel(output_file, index=False)

print(f"Processed node data saved to '{output_file}'")

[DEBUG] 모든 노드 데이터:
   Node ID Node Type Label Sentiment  Frequency Source Nodes
0  별로에요_부정     연관 노드  별로에요        부정          1           []
1      냄새가     중심 노드   냄새가       무감정         63           []
2     쎄_부정     연관 노드     쎄        부정          1           []
3       향은     중심 노드    향은       무감정         35           []
4    향이_부정     연관 노드    향이        부정          5           []
5     향수같이     중심 노드  향수같이       무감정          5           []
6  분홍이가_부정     연관 노드  분홍이가        부정          1           []
7  쓰기에는_부정     연관 노드  쓰기에는        부정          1           []
8       향이     중심 노드    향이       무감정        173           []
9  강합니다_부정     연관 노드  강합니다        부정          1           []
[DEBUG] 모든 엣지 데이터:
Source: 냄새가, Target: 별로에요_부정
Source: 향은, Target: 쎄_부정
Source: 향수같이, Target: 향이_부정
Source: 향은, Target: 분홍이가_부정
Source: 향이, Target: 쓰기에는_부정
Source: 향이, Target: 강합니다_부정
Source: 향이, Target: 세럼_부정
Source: 향이네요, Target: 놓는_부정
Source: 무향을, Target: 선호하는_부정
Source: 약냄새같기도하고, Target: 향이_부정
Source: 향이

# 트러블

## 긍부정

In [80]:
from pyvis.network import Network
import pandas as pd
import re
from collections import Counter
from konlpy.tag import Okt

# 형태소 분석기 초기화
okt = Okt()

# 불용어 리스트 정의 (한국어 불용어)
with open(r"\Users\kyn03\OneDrive\바탕 화면\project_file\stopwords-ko(한국어불용어).txt", encoding='utf-8') as f:
    stopwords = f.read().splitlines()

# 추가 불용어 정의
stopwords += ['좀', '것', '달바', '미스트', '스프레이', '1으로', '1이라', '1에', '너무', '느낌', '잘', '그냥', '많이', 
              '있어서', '같아요', '스프레이가', '미스트가', '미스트는', '약간의', '생각보다', '있으며', '살짝', '같아서', 
              '느낌이', '저는', '진짜', '엄청', '합니다', '더', '안', '느낌은', '효과는', '조금', '분들은', '크림미스트라서',
              '크림미스트라', '크림', '크림이', '미스트라', '미스트라서', '그런지', '못', '이거', '완전', '아벤느', '다', '개', '걸',
              '매우','음','확','전','제','달바는','막','넘','너무너무','특히','근데','취향이','정말','덜','뭔가',
              '저의','굉장히','아니지만','아니고','해야할까요','제품','시',
              '있는데','샀는데','아니라','기분','썼을것같은데','바꿔봤는데','그런','사봤는데','해야할까요','저에게는',
              '하는데','아주','알겠더라구용','되게','나요','나서','나면서','진심','나니까','저한텐','저한테는','구매했어요',]

# 데이터 로드
data = pd.read_excel(r"\Users\kyn03\OneDrive\바탕 화면\project_file\sorted_merged_dalba.xlsx")
data = data.where(pd.notna(data), None)  # 결측치 처리
data = data.reset_index()

# 인덱스 넘버가 3 + 7 * n인 경우 필터링하여 'Aspect2 Mapped Labels' 행 필터링
filtered_data = data.iloc[[i for i in range(len(data)) if (i - 2) % 7 == 0]]

# 키워드 설정
aspect_keywords = ["트러블", "여드름"]

# 텍스트 전처리 함수
def preprocess_text(text):
    """텍스트에서 특수문자 제거 및 불용어 필터링"""
    cleaned_text = re.sub(r"[^가-힣0-9\s]", "", text)
    tokens = [word for word in cleaned_text.split() if word not in stopwords]
    return tokens

# 네트워크 그래프 생성을 위한 데이터 수집
nodes_data = set()  # 중복 방지를 위해 set 사용
all_previous_tokens = []

# 키워드와 관련된 단어 연결
for idx, row in filtered_data.iterrows():
    original_idx = data[data['index'] == row['index']].index[0]
    tokenized_text_index = original_idx + 3  # 'Tokenized Text' 행
    sentiment_row_index = tokenized_text_index - 5  # 감정 데이터는 동일 열의 -5행에 위치

    if tokenized_text_index < len(data):

        for col_index in range(2, len(data.columns)):  # 각 열을 순회
            tokenized_text = str(data.iat[tokenized_text_index, col_index])

            # 감정 데이터 가져오기
            try:
                sentiment_value = str(data.iat[sentiment_row_index, col_index]).strip()
            except Exception as e:
                print(f"[ERROR] 감정 데이터를 가져오는 중 에러 발생: {e}")
                sentiment_value = None

            # Null 또는 빈 문자열 확인
            if not tokenized_text or tokenized_text.strip() == "":
                continue

            # 키워드 포함 여부 확인
            if not any(keyword in tokenized_text for keyword in aspect_keywords):
                continue

            # 키워드가 포함된 경우 처리
            print(f"[INFO] 키워드 포함 텍스트: {tokenized_text}, 감정: {sentiment_value}")

            # 기준 셀의 왼쪽과 오른쪽 셀 값을 추출
            left_text = (
                str(data.iat[tokenized_text_index, col_index - 1])
                if col_index > 2  # 왼쪽 셀이 존재하는 경우
                else None
            )
            right_text = (
                str(data.iat[tokenized_text_index, col_index + 1])
                if col_index < len(data.columns) - 1  # 오른쪽 셀이 존재하는 경우
                else None
            )

            # 제외할 접미사를 리스트로 관리
            EXCLUDED_SUFFIXES = ("-")
            EXCLUDED_KEYWORDS = ["-"]
            # 왼쪽 셀 추가
            if left_text and left_text.strip():
                stripped_left_text = left_text.strip()  # 공백 제거
                # 접미사 검사
                if any(stripped_left_text.endswith(suffix) for suffix in EXCLUDED_SUFFIXES):
                    print(f"[INFO] 왼쪽 셀이 '{EXCLUDED_SUFFIXES}' 중 하나로 끝나서 제외됨: {stripped_left_text}")

                elif any(keyword in stripped_left_text for keyword in EXCLUDED_KEYWORDS):
                    print(f"[INFO] 왼쪽 셀이 제외 키워드를 포함하여 제외됨: {stripped_left_text}")
                else:
                    nodes_data.add((tokenized_text, stripped_left_text, tokenized_text_index, col_index))  # set에 추가
                    all_previous_tokens.append(stripped_left_text)

            # 오른쪽 셀 추가
            if right_text and right_text.strip():
                stripped_right_text = right_text.strip()  # 공백 제거
                # 키워드 포함 여부 확인
                if any(keyword in stripped_right_text for keyword in EXCLUDED_KEYWORDS):
                    print(f"[INFO] 오른쪽 셀이 제외 키워드를 포함하여 제외됨: {stripped_right_text}")
                else:
                    nodes_data.add((tokenized_text, right_text.strip(),tokenized_text_index,col_index))  # set에 추가
                    all_previous_tokens.append(right_text.strip())

 

# 빈도수 카운터 생성
if not all_previous_tokens:
    print("[ERROR] 관련된 서술어가 없습니다. 전처리 또는 데이터 로드 로직을 확인하세요.")
else:
    # 감정별로 빈도수를 따로 저장하기 위한 딕셔너리
    sentiment_frequency = {
        "긍정": Counter(),
        "부정": Counter(),
        "중립": Counter()
    }

    # 전체 빈도수 저장
    total_frequency = Counter()

    # 빈도수 계산
    for tokenized_text, related_text, tokenized_text_index, col_index in nodes_data:
        # 감정 값 가져오기
        try:
            sentiment_value = str(data.iat[tokenized_text_index - 5, col_index]).strip()
            # B-긍정, I-긍정 같은 값을 "긍정"으로 통합
            if sentiment_value.startswith("B-") or sentiment_value.startswith("I-"):
                sentiment_value = sentiment_value[2:]
        except Exception as e:
            print(f"[ERROR] 감정 데이터를 가져오는 중 에러 발생: {e}")
            sentiment_value = None

        # 유효한 감정 값인지 확인
        if sentiment_value in sentiment_frequency:
            sentiment_frequency[sentiment_value][related_text] += 1
            total_frequency[related_text] += 1  # 전체 빈도수 업데이트



[INFO] 키워드 포함 텍스트: 트러블이, 감정: B-긍정
[INFO] 키워드 포함 텍스트: 여드름에, 감정: I-부정
[INFO] 키워드 포함 텍스트: 여드름, 감정: I-부정
[INFO] 키워드 포함 텍스트: 여드름에, 감정: I-부정
[INFO] 키워드 포함 텍스트: 여드름, 감정: I-부정
[INFO] 키워드 포함 텍스트: 여드름에, 감정: I-부정
[INFO] 키워드 포함 텍스트: 여드름, 감정: I-부정
[INFO] 키워드 포함 텍스트: 트러블도, 감정: B-긍정
[INFO] 키워드 포함 텍스트: 여드름에, 감정: I-부정
[INFO] 키워드 포함 텍스트: 여드름, 감정: I-부정
[INFO] 키워드 포함 텍스트: 여드름에, 감정: I-부정
[INFO] 키워드 포함 텍스트: 여드름, 감정: I-부정
[INFO] 키워드 포함 텍스트: 여드름에, 감정: I-부정
[INFO] 키워드 포함 텍스트: 여드름, 감정: I-부정
[INFO] 키워드 포함 텍스트: 여드름에, 감정: I-부정
[INFO] 키워드 포함 텍스트: 여드름, 감정: I-부정
[INFO] 키워드 포함 텍스트: 트러블도, 감정: B-긍정
[INFO] 키워드 포함 텍스트: 여드름피부라, 감정: O
[INFO] 키워드 포함 텍스트: 트러블, 감정: B-긍정
[INFO] 키워드 포함 텍스트: 여드름, 감정: B-긍정
[INFO] 키워드 포함 텍스트: 여드름, 감정: B-긍정
[INFO] 키워드 포함 텍스트: 피부트러블이, 감정: O
[INFO] 키워드 포함 텍스트: 트러블안나고, 감정: B-긍정
[INFO] 키워드 포함 텍스트: 트러블나는게, 감정: O
[INFO] 키워드 포함 텍스트: 트러블이, 감정: B-부정
[INFO] 키워드 포함 텍스트: 트러블, 감정: B-긍정
[INFO] 키워드 포함 텍스트: 트러블을, 감정: I-긍정
[INFO] 키워드 포함 텍스트: 트러블도, 감정: B-긍정
[INFO] 키워드 포함 텍스트: 여드름같이, 감정: I-부정
[INFO] 키워드 포함 텍스트: 여드름이, 

In [81]:
# 네트워크 그래프 생성
net = Network(notebook=True, cdn_resources="in_line")
net.toggle_physics(True)

# 중심 노드와 연관 노드 간 연결 정보 기록
connected_nodes = {}
center_frequency = Counter()

# 그래프 노드 및 엣지 추가
EXCLUDED_MAIN_KEYWORDS = ["-"]

for main_token, related_token, sentiment_row_index, col_index in nodes_data:
    main_node_key = main_token.strip()  # 중심 노드 키 생성
    print(f"[DEBUG] Processing pair: main_token={main_token}, related_token={related_token}")

    # 유효성 검증 및 불용어 필터링
    if not re.match(r"^[가-힣]+$", main_node_key) or not re.match(r"^[가-힣]+$", related_token):
        print(f"[INFO] main_token 또는 related_token이 유효하지 않음: {main_token}, {related_token}")
        continue
    if main_token in stopwords or related_token in stopwords:
        print(f"[INFO] 불용어로 제외됨: {main_token} 또는 {related_token}")
        continue
    if any(keyword in main_node_key for keyword in EXCLUDED_MAIN_KEYWORDS):
        print(f"[INFO] 중심 노드가 제외 키워드를 포함하여 추가되지 않음: {main_node_key}")
        continue

    # 감정 값 가져오기
    related_token_sentiment = None
    try:
        related_token_sentiment = str(data.iat[sentiment_row_index - 5, col_index]).strip()
        if related_token_sentiment.startswith("B-") or related_token_sentiment.startswith("I-"):
            related_token_sentiment = related_token_sentiment[2:]  # 통합
    except Exception as e:
        print(f"[ERROR] related_token 감정 데이터 가져오기 실패: {e}")
        continue

    # 연관 노드 추가
    related_token_frequency = sentiment_frequency.get(related_token_sentiment, {}).get(related_token, 0)
    if related_token_frequency >= 1:
        if "중립" in related_token_sentiment or "O" in related_token_sentiment:
            print(f"[INFO] 연관 노드가 '중립' 또는 'O' 감정으로 추가되지 않음: {related_token}")
            continue

        related_node_key = f"{related_token}_{related_token_sentiment}"
        if related_node_key not in net.get_nodes():
            node_size = min(15 + related_token_frequency * 2, 50)
            related_token_color = {
                "긍정": "#6EC664",
                "중립": "#BDC3C7",
                "부정": "#E74C3C"
            }.get(related_token_sentiment, "black")
            net.add_node(
                related_node_key,
                label=related_token,
                color=related_token_color,
                size=node_size,
                title=f"{related_token} (빈도: {related_token_frequency})"
            )
            print(f"[DEBUG] 추가된 연관 노드: {related_node_key}")

        # 엣지 및 중심 노드 추가
        if main_node_key not in net.get_nodes():
            net.add_node(
                main_node_key,
                label=main_token,
                color={"background": "white", "border": "black"},
                size=25,
                title=f"{main_token} (빈도: 0)"  # 빈도수는 추후 갱신
            )
        # 엣지 추가
        net.add_edge(main_node_key, related_node_key, color=related_token_color)
        print(f"[DEBUG] 엣지 추가됨: {main_node_key} -> {related_node_key}")
        connected_nodes.setdefault(main_node_key, []).append(related_node_key)

# 중심 노드 빈도 계산 (화면에 표시된 연관 노드 기반)
center_frequency.clear()  # 기존 빈도수 초기화
for main_node_key, related_nodes in connected_nodes.items():
    for related_node_key in related_nodes:
        related_token = related_node_key.split("_")[0]
        related_token_sentiment = related_node_key.split("_")[1]
        related_token_frequency = sentiment_frequency.get(related_token_sentiment, {}).get(related_token, 0)
        center_frequency[main_node_key] += related_token_frequency
        print(f"[DEBUG] 중심 노드 '{main_node_key}'에 '{related_token}'의 빈도 {related_token_frequency} 추가됨")

# 중심 노드 빈도 업데이트 (화면에 표시된 값으로 갱신)
for node in net.nodes:
    if node["id"] in center_frequency:
        node["title"] = f"{node['id']} (빈도: {center_frequency[node['id']]})"
        print(f"[DEBUG] 중심 노드 '{node['id']}'의 빈도 갱신됨: {center_frequency[node['id']]}")

# 디버깅: 최종 중심 노드 빈도수 출력
print("[DEBUG] 최종 중심 노드 빈도수:")
for node, freq in center_frequency.items():
    print(f"{node}: {freq}")

# 네트워크 그래프 시각화
net.show("acne_positive_negative.html")


[DEBUG] Processing pair: main_token=여드름이나, related_token=좋아요
[DEBUG] 추가된 연관 노드: 좋아요_부정
[DEBUG] 엣지 추가됨: 여드름이나 -> 좋아요_부정
[DEBUG] Processing pair: main_token=트러블이, related_token=올라오지
[DEBUG] Processing pair: main_token=트러블, related_token=역시나
[DEBUG] Processing pair: main_token=트러블이, related_token=너무
[INFO] 불용어로 제외됨: 트러블이 또는 너무
[DEBUG] Processing pair: main_token=트러블, related_token=폭발해서
[DEBUG] 추가된 연관 노드: 폭발해서_긍정
[DEBUG] 엣지 추가됨: 트러블 -> 폭발해서_긍정
[DEBUG] Processing pair: main_token=트러블도, related_token=없고
[DEBUG] 추가된 연관 노드: 없고_긍정
[DEBUG] 엣지 추가됨: 트러블도 -> 없고_긍정
[DEBUG] Processing pair: main_token=트러블, related_token=올라왔어요ㅜㅜ
[INFO] main_token 또는 related_token이 유효하지 않음: 트러블, 올라왔어요ㅜㅜ
[DEBUG] Processing pair: main_token=트러블, related_token=피부에는
[DEBUG] 추가된 연관 노드: 피부에는_부정
[DEBUG] 엣지 추가됨: 트러블 -> 피부에는_부정
[DEBUG] Processing pair: main_token=트러블은, related_token=안올라왔어요
[DEBUG] 추가된 연관 노드: 안올라왔어요_긍정
[DEBUG] 엣지 추가됨: 트러블은 -> 안올라왔어요_긍정
[DEBUG] Processing pair: main_token=여드름성, related_token=피부
[DEBUG] 추가된 연관 노드:

In [77]:
import pandas as pd
import re

# 네트워크에 표시된 모든 노드 데이터 가져오기
all_nodes_data = []
for node in net.nodes:
    # 노드 정보 추출
    node_id = node["id"]
    node_label = node.get("label", "")
    node_color = node.get("color", "")
    node_title = node.get("title", "")  # 빈도수 정보 포함 가능

    # 노드 종류 결정
    if "_" in node_id:  # "_"이 포함된 경우 연관 노드
        node_type = "연관 노드"
    else:  # "_"이 없는 경우 중심 노드
        node_type = "중심 노드"

    # 감정 추출 (색상 기반)
    if node_color == "#6EC664":  # 연한 녹색
        node_sentiment = "긍정"
    elif node_color == "#BDC3C7":  # 연한 회색 (중립)
        node_sentiment = "중립"
    elif node_color == "#E74C3C":  # 연한 빨간색 (부정)
        node_sentiment = "부정"
    else:
        node_sentiment = "무감정"

    # 빈도수 추출 (Title에서 추출)
    try:
        node_frequency = int(re.search(r"\(빈도: (\d+)\)", node_title).group(1))
    except AttributeError:
        node_frequency = 0  # 빈도수 정보가 없으면 0으로 설정

    # 데이터 추가
    all_nodes_data.append({
        "Node ID": node_id,
        "Node Type": node_type,
        "Label": node_label,
        "Sentiment": node_sentiment,
        "Frequency": node_frequency,
        "Source Nodes": []  # 초기값으로 빈 리스트 설정 (연관 노드일 경우 추가적으로 설정)
    })

# DataFrame으로 변환
all_nodes_df = pd.DataFrame(all_nodes_data)

# 디버깅: 모든 노드 출력
print("[DEBUG] 모든 노드 데이터:")
print(all_nodes_df.head(10))

# 디버깅: 모든 엣지 데이터 출력
print("[DEBUG] 모든 엣지 데이터:")
for edge in net.edges:
    print(f"Source: {edge['from']}, Target: {edge['to']}")

# 엣지 데이터를 활용하여 Source Nodes 설정
for edge in net.edges:
    source = edge["from"]
    target = edge["to"]

    # 디버깅: 현재 처리 중인 엣지
    print(f"[DEBUG] 처리 중인 엣지: Source={source}, Target={target}")

    # 타겟 노드가 연관 노드인 경우 Source Nodes에 소스 노드 추가
    if "_" in target:  # 타겟 노드가 연관 노드
        target_idx = all_nodes_df[all_nodes_df["Node ID"] == target].index
        if not target_idx.empty:
            current_sources = all_nodes_df.at[target_idx[0], "Source Nodes"]
            if isinstance(current_sources, list):  # 리스트 형태인 경우
                current_sources.append(source)
            else:  # 초기값이 없거나 비어 있을 경우
                current_sources = [source]
            # Source Nodes 갱신
            all_nodes_df.at[target_idx[0], "Source Nodes"] = current_sources

# Source Nodes를 문자열로 변환 (콤마로 연결)
all_nodes_df["Source Nodes"] = all_nodes_df["Source Nodes"].apply(
    lambda x: ", ".join(x) if isinstance(x, list) else ""
)

# 정렬: 중심 노드 기준, 빈도수 기준 내림차순
all_nodes_df = all_nodes_df.sort_values(by=["Node Type", "Frequency"], ascending=[True, False])

# 결과 저장
output_file = "network_nodes_data_acne_positive_negative.xlsx"
all_nodes_df.to_excel(output_file, index=False)

print(f"Processed node data saved to '{output_file}'")

[DEBUG] 모든 노드 데이터:
     Node ID Node Type   Label Sentiment  Frequency Source Nodes
0     좋아요_부정     연관 노드     좋아요        부정          1           []
1      여드름이나     중심 노드   여드름이나       무감정          2           []
2    폭발해서_긍정     연관 노드    폭발해서        긍정          1           []
3        트러블     중심 노드     트러블       무감정        222           []
4      없고_긍정     연관 노드      없고        긍정         10           []
5       트러블도     중심 노드    트러블도       무감정        181           []
6    피부에는_부정     연관 노드    피부에는        부정          9           []
7  안올라왔어요_긍정     연관 노드  안올라왔어요        긍정          1           []
8       트러블은     중심 노드    트러블은       무감정         25           []
9      피부_긍정     연관 노드      피부        긍정          7           []
[DEBUG] 모든 엣지 데이터:
Source: 여드름이나, Target: 좋아요_부정
Source: 트러블, Target: 폭발해서_긍정
Source: 트러블도, Target: 없고_긍정
Source: 트러블, Target: 피부에는_부정
Source: 트러블은, Target: 안올라왔어요_긍정
Source: 여드름성, Target: 피부_긍정
Source: 트러블, Target: 올라와서_긍정
Source: 트러블, Target: 전혀_긍정
Source: 트러블, Ta

# 오일(성분)

## 긍부정

In [82]:
from pyvis.network import Network
import pandas as pd
import re
from collections import Counter
from konlpy.tag import Okt

# 형태소 분석기 초기화
okt = Okt()

# 불용어 리스트 정의 (한국어 불용어)
with open(r"\Users\kyn03\OneDrive\바탕 화면\project_file\stopwords-ko(한국어불용어).txt", encoding='utf-8') as f:
    stopwords = f.read().splitlines()

# 추가 불용어 정의
stopwords += ['좀', '것', '달바', '미스트', '스프레이', '1으로', '1이라', '1에', '너무', '느낌', '잘', '그냥', '많이', 
              '있어서', '같아요', '스프레이가', '미스트가', '미스트는', '약간의', '생각보다', '있으며', '살짝', '같아서', 
              '느낌이', '저는', '진짜', '엄청', '합니다', '더', '안', '느낌은', '효과는', '조금', '분들은', '크림미스트라서',
              '크림미스트라', '크림', '크림이', '미스트라', '미스트라서', '그런지', '못', '이거', '완전', '아벤느', '다', '개', '걸',
              '매우','음','확','전','제','달바는','막','넘','너무너무','특히','근데','취향이','정말','덜','뭔가',
              '저의','굉장히','아니지만','아니고','해야할까요','제품','시',
              '있는데','샀는데','아니라','기분','썼을것같은데','바꿔봤는데','그런','사봤는데','해야할까요','저에게는',
              '하는데','아주','알겠더라구용','되게','나요','나서','나면서','진심','나니까','저한텐','저한테는','구매했어요',]

# 데이터 로드
data = pd.read_excel(r"\Users\kyn03\OneDrive\바탕 화면\project_file\sorted_merged_dalba.xlsx")
data = data.where(pd.notna(data), None)  # 결측치 처리
data = data.reset_index()

# 인덱스 넘버가 3 + 7 * n인 경우 필터링하여 'Aspect2 Mapped Labels' 행 필터링
filtered_data = data.iloc[[i for i in range(len(data)) if (i - 2) % 7 == 0]]

# 키워드 설정
aspect_keywords = ["오일", "성분"]

# 텍스트 전처리 함수
def preprocess_text(text):
    """텍스트에서 특수문자 제거 및 불용어 필터링"""
    cleaned_text = re.sub(r"[^가-힣0-9\s]", "", text)
    tokens = [word for word in cleaned_text.split() if word not in stopwords]
    return tokens

# 네트워크 그래프 생성을 위한 데이터 수집
nodes_data = set()  # 중복 방지를 위해 set 사용
all_previous_tokens = []

# 키워드와 관련된 단어 연결
for idx, row in filtered_data.iterrows():
    original_idx = data[data['index'] == row['index']].index[0]
    tokenized_text_index = original_idx + 3  # 'Tokenized Text' 행
    sentiment_row_index = tokenized_text_index - 5  # 감정 데이터는 동일 열의 -5행에 위치

    if tokenized_text_index < len(data):

        for col_index in range(2, len(data.columns)):  # 각 열을 순회
            tokenized_text = str(data.iat[tokenized_text_index, col_index])

            # 감정 데이터 가져오기
            try:
                sentiment_value = str(data.iat[sentiment_row_index, col_index]).strip()
            except Exception as e:
                print(f"[ERROR] 감정 데이터를 가져오는 중 에러 발생: {e}")
                sentiment_value = None

            # Null 또는 빈 문자열 확인
            if not tokenized_text or tokenized_text.strip() == "":
                continue

            # 키워드 포함 여부 확인
            if not any(keyword in tokenized_text for keyword in aspect_keywords):
                continue

            # 키워드가 포함된 경우 처리
            print(f"[INFO] 키워드 포함 텍스트: {tokenized_text}, 감정: {sentiment_value}")

            # 기준 셀의 왼쪽과 오른쪽 셀 값을 추출
            left_text = (
                str(data.iat[tokenized_text_index, col_index - 1])
                if col_index > 2  # 왼쪽 셀이 존재하는 경우
                else None
            )
            right_text = (
                str(data.iat[tokenized_text_index, col_index + 1])
                if col_index < len(data.columns) - 1  # 오른쪽 셀이 존재하는 경우
                else None
            )

            # 제외할 접미사를 리스트로 관리
            EXCLUDED_SUFFIXES = ("-")
            EXCLUDED_KEYWORDS = ["-"]
            # 왼쪽 셀 추가
            if left_text and left_text.strip():
                stripped_left_text = left_text.strip()  # 공백 제거
                # 접미사 검사
                if any(stripped_left_text.endswith(suffix) for suffix in EXCLUDED_SUFFIXES):
                    print(f"[INFO] 왼쪽 셀이 '{EXCLUDED_SUFFIXES}' 중 하나로 끝나서 제외됨: {stripped_left_text}")

                elif any(keyword in stripped_left_text for keyword in EXCLUDED_KEYWORDS):
                    print(f"[INFO] 왼쪽 셀이 제외 키워드를 포함하여 제외됨: {stripped_left_text}")
                else:
                    nodes_data.add((tokenized_text, stripped_left_text, tokenized_text_index, col_index))  # set에 추가
                    all_previous_tokens.append(stripped_left_text)

            # 오른쪽 셀 추가
            if right_text and right_text.strip():
                stripped_right_text = right_text.strip()  # 공백 제거
                # 키워드 포함 여부 확인
                if any(keyword in stripped_right_text for keyword in EXCLUDED_KEYWORDS):
                    print(f"[INFO] 오른쪽 셀이 제외 키워드를 포함하여 제외됨: {stripped_right_text}")
                else:
                    nodes_data.add((tokenized_text, right_text.strip(),tokenized_text_index,col_index))  # set에 추가
                    all_previous_tokens.append(right_text.strip())

 

# 빈도수 카운터 생성
if not all_previous_tokens:
    print("[ERROR] 관련된 서술어가 없습니다. 전처리 또는 데이터 로드 로직을 확인하세요.")
else:
    # 감정별로 빈도수를 따로 저장하기 위한 딕셔너리
    sentiment_frequency = {
        "긍정": Counter(),
        "부정": Counter(),
        "중립": Counter()
    }

    # 전체 빈도수 저장
    total_frequency = Counter()

    # 빈도수 계산
    for tokenized_text, related_text, tokenized_text_index, col_index in nodes_data:
        # 감정 값 가져오기
        try:
            sentiment_value = str(data.iat[tokenized_text_index - 5, col_index]).strip()
            # B-긍정, I-긍정 같은 값을 "긍정"으로 통합
            if sentiment_value.startswith("B-") or sentiment_value.startswith("I-"):
                sentiment_value = sentiment_value[2:]
        except Exception as e:
            print(f"[ERROR] 감정 데이터를 가져오는 중 에러 발생: {e}")
            sentiment_value = None

        # 유효한 감정 값인지 확인
        if sentiment_value in sentiment_frequency:
            sentiment_frequency[sentiment_value][related_text] += 1
            total_frequency[related_text] += 1  # 전체 빈도수 업데이트


[INFO] 키워드 포함 텍스트: 성분, 감정: I-긍정
[INFO] 키워드 포함 텍스트: 오일, 감정: O
[INFO] 키워드 포함 텍스트: 성분의, 감정: I-부정
[INFO] 키워드 포함 텍스트: 오일, 감정: O
[INFO] 키워드 포함 텍스트: 성분의, 감정: I-부정
[INFO] 키워드 포함 텍스트: 오일, 감정: O
[INFO] 키워드 포함 텍스트: 성분의, 감정: I-부정
[INFO] 키워드 포함 텍스트: 성분, 감정: I-긍정
[INFO] 키워드 포함 텍스트: 성분으로, 감정: I-긍정
[INFO] 키워드 포함 텍스트: 성분이, 감정: I-긍정
[INFO] 키워드 포함 텍스트: 성분이, 감정: I-긍정
[INFO] 키워드 포함 텍스트: 성분이, 감정: I-긍정
[INFO] 키워드 포함 텍스트: 성분, 감정: I-긍정
[INFO] 키워드 포함 텍스트: 오일, 감정: I-긍정
[INFO] 키워드 포함 텍스트: 오일이, 감정: I-긍정
[INFO] 키워드 포함 텍스트: 성분을, 감정: I-긍정
[INFO] 키워드 포함 텍스트: 오일은, 감정: I-긍정
[INFO] 키워드 포함 텍스트: 오일, 감정: O
[INFO] 키워드 포함 텍스트: 오일, 감정: O
[INFO] 키워드 포함 텍스트: 성분을, 감정: O
[INFO] 키워드 포함 텍스트: 성분, 감정: I-긍정
[INFO] 키워드 포함 텍스트: 성분, 감정: I-긍정
[INFO] 키워드 포함 텍스트: 오일, 감정: O
[INFO] 키워드 포함 텍스트: 성분의, 감정: I-부정
[INFO] 키워드 포함 텍스트: 성분이, 감정: I-긍정
[INFO] 키워드 포함 텍스트: 성분이, 감정: I-긍정
[INFO] 키워드 포함 텍스트: 성분이, 감정: I-긍정
[INFO] 키워드 포함 텍스트: 오일, 감정: O
[INFO] 키워드 포함 텍스트: 성분의, 감정: O
[INFO] 키워드 포함 텍스트: 오일, 감정: O
[INFO] 키워드 포함 텍스트: 성분의, 감정: I-부정
[INFO] 키워드 포함 텍스트: 오

In [83]:
# 네트워크 그래프 생성
net = Network(notebook=True, cdn_resources="in_line")
net.toggle_physics(True)

# 중심 노드와 연관 노드 간 연결 정보 기록
connected_nodes = {}
center_frequency = Counter()

# 그래프 노드 및 엣지 추가
EXCLUDED_MAIN_KEYWORDS = ["지성", "건성"]

for main_token, related_token, sentiment_row_index, col_index in nodes_data:
    main_node_key = main_token.strip()  # 중심 노드 키 생성
    print(f"[DEBUG] Processing pair: main_token={main_token}, related_token={related_token}")

    # 유효성 검증 및 불용어 필터링
    if not re.match(r"^[가-힣]+$", main_node_key) or not re.match(r"^[가-힣]+$", related_token):
        print(f"[INFO] main_token 또는 related_token이 유효하지 않음: {main_token}, {related_token}")
        continue
    if main_token in stopwords or related_token in stopwords:
        print(f"[INFO] 불용어로 제외됨: {main_token} 또는 {related_token}")
        continue
    if any(keyword in main_node_key for keyword in EXCLUDED_MAIN_KEYWORDS):
        print(f"[INFO] 중심 노드가 제외 키워드를 포함하여 추가되지 않음: {main_node_key}")
        continue

    # 감정 값 가져오기
    related_token_sentiment = None
    try:
        related_token_sentiment = str(data.iat[sentiment_row_index - 5, col_index]).strip()
        if related_token_sentiment.startswith("B-") or related_token_sentiment.startswith("I-"):
            related_token_sentiment = related_token_sentiment[2:]  # 통합
    except Exception as e:
        print(f"[ERROR] related_token 감정 데이터 가져오기 실패: {e}")
        continue

    # 연관 노드 추가
    related_token_frequency = sentiment_frequency.get(related_token_sentiment, {}).get(related_token, 0)
    if related_token_frequency >= 1:
        if "중립" in related_token_sentiment or "O" in related_token_sentiment:
            print(f"[INFO] 연관 노드가 '중립' 또는 'O' 감정으로 추가되지 않음: {related_token}")
            continue

        related_node_key = f"{related_token}_{related_token_sentiment}"
        if related_node_key not in net.get_nodes():
            node_size = min(15 + related_token_frequency * 2, 50)
            related_token_color = {
                "긍정": "#6EC664",
                "중립": "#BDC3C7",
                "부정": "#E74C3C"
            }.get(related_token_sentiment, "black")
            net.add_node(
                related_node_key,
                label=related_token,
                color=related_token_color,
                size=node_size,
                title=f"{related_token} (빈도: {related_token_frequency})"
            )
            print(f"[DEBUG] 추가된 연관 노드: {related_node_key}")

        # 엣지 및 중심 노드 추가
        if main_node_key not in net.get_nodes():
            net.add_node(
                main_node_key,
                label=main_token,
                color={"background": "white", "border": "black"},
                size=25,
                title=f"{main_token} (빈도: 0)"  # 빈도수는 추후 갱신
            )
        # 엣지 추가
        net.add_edge(main_node_key, related_node_key, color=related_token_color)
        print(f"[DEBUG] 엣지 추가됨: {main_node_key} -> {related_node_key}")
        connected_nodes.setdefault(main_node_key, []).append(related_node_key)

# 중심 노드 빈도 계산 (화면에 표시된 연관 노드 기반)
center_frequency.clear()  # 기존 빈도수 초기화
for main_node_key, related_nodes in connected_nodes.items():
    for related_node_key in related_nodes:
        related_token = related_node_key.split("_")[0]
        related_token_sentiment = related_node_key.split("_")[1]
        related_token_frequency = sentiment_frequency.get(related_token_sentiment, {}).get(related_token, 0)
        center_frequency[main_node_key] += related_token_frequency
        print(f"[DEBUG] 중심 노드 '{main_node_key}'에 '{related_token}'의 빈도 {related_token_frequency} 추가됨")

# 중심 노드 빈도 업데이트 (화면에 표시된 값으로 갱신)
for node in net.nodes:
    if node["id"] in center_frequency:
        node["title"] = f"{node['id']} (빈도: {center_frequency[node['id']]})"
        print(f"[DEBUG] 중심 노드 '{node['id']}'의 빈도 갱신됨: {center_frequency[node['id']]}")

# 디버깅: 최종 중심 노드 빈도수 출력
print("[DEBUG] 최종 중심 노드 빈도수:")
for node, freq in center_frequency.items():
    print(f"{node}: {freq}")

# 네트워크 그래프 시각화
net.show("oil_positive_negative.html")


[DEBUG] Processing pair: main_token=오일리하지, related_token=않아서
[DEBUG] 추가된 연관 노드: 않아서_긍정
[DEBUG] 엣지 추가됨: 오일리하지 -> 않아서_긍정
[DEBUG] Processing pair: main_token=오일리하지, related_token=않은
[DEBUG] 추가된 연관 노드: 않은_긍정
[DEBUG] 엣지 추가됨: 오일리하지 -> 않은_긍정
[DEBUG] Processing pair: main_token=오일, related_token=세럼
[DEBUG] 추가된 연관 노드: 세럼_긍정
[DEBUG] 엣지 추가됨: 오일 -> 세럼_긍정
[DEBUG] Processing pair: main_token=오일이, related_token=구매했어요
[INFO] 불용어로 제외됨: 오일이 또는 구매했어요
[DEBUG] Processing pair: main_token=오일이라고하네요, related_token=.
[INFO] main_token 또는 related_token이 유효하지 않음: 오일이라고하네요, .
[DEBUG] Processing pair: main_token=오일덕분인지, related_token=있는
[DEBUG] 추가된 연관 노드: 있는_부정
[DEBUG] 엣지 추가됨: 오일덕분인지 -> 있는_부정
[DEBUG] Processing pair: main_token=오일, related_token=성분이
[DEBUG] 추가된 연관 노드: 성분이_긍정
[DEBUG] 엣지 추가됨: 오일 -> 성분이_긍정
[DEBUG] Processing pair: main_token=오일이, related_token=섞여
[DEBUG] 추가된 연관 노드: 섞여_긍정
[DEBUG] 엣지 추가됨: 오일이 -> 섞여_긍정
[DEBUG] Processing pair: main_token=오일리하고, related_token=첨엔
[DEBUG] 추가된 연관 노드: 첨엔_부정
[DEBUG] 엣지 추가됨: 오

In [84]:
import pandas as pd
import re

# 네트워크에 표시된 모든 노드 데이터 가져오기
all_nodes_data = []
for node in net.nodes:
    # 노드 정보 추출
    node_id = node["id"]
    node_label = node.get("label", "")
    node_color = node.get("color", "")
    node_title = node.get("title", "")  # 빈도수 정보 포함 가능

    # 노드 종류 결정
    if "_" in node_id:  # "_"이 포함된 경우 연관 노드
        node_type = "연관 노드"
    else:  # "_"이 없는 경우 중심 노드
        node_type = "중심 노드"

    # 감정 추출 (색상 기반)
    if node_color == "#6EC664":  # 연한 녹색
        node_sentiment = "긍정"
    elif node_color == "#BDC3C7":  # 연한 회색 (중립)
        node_sentiment = "중립"
    elif node_color == "#E74C3C":  # 연한 빨간색 (부정)
        node_sentiment = "부정"
    else:
        node_sentiment = "무감정"

    # 빈도수 추출 (Title에서 추출)
    try:
        node_frequency = int(re.search(r"\(빈도: (\d+)\)", node_title).group(1))
    except AttributeError:
        node_frequency = 0  # 빈도수 정보가 없으면 0으로 설정

    # 데이터 추가
    all_nodes_data.append({
        "Node ID": node_id,
        "Node Type": node_type,
        "Label": node_label,
        "Sentiment": node_sentiment,
        "Frequency": node_frequency,
        "Source Nodes": []  # 초기값으로 빈 리스트 설정 (연관 노드일 경우 추가적으로 설정)
    })

# DataFrame으로 변환
all_nodes_df = pd.DataFrame(all_nodes_data)

# 디버깅: 모든 노드 출력
print("[DEBUG] 모든 노드 데이터:")
print(all_nodes_df.head(10))

# 디버깅: 모든 엣지 데이터 출력
print("[DEBUG] 모든 엣지 데이터:")
for edge in net.edges:
    print(f"Source: {edge['from']}, Target: {edge['to']}")

# 엣지 데이터를 활용하여 Source Nodes 설정
for edge in net.edges:
    source = edge["from"]
    target = edge["to"]

    # 디버깅: 현재 처리 중인 엣지
    print(f"[DEBUG] 처리 중인 엣지: Source={source}, Target={target}")

    # 타겟 노드가 연관 노드인 경우 Source Nodes에 소스 노드 추가
    if "_" in target:  # 타겟 노드가 연관 노드
        target_idx = all_nodes_df[all_nodes_df["Node ID"] == target].index
        if not target_idx.empty:
            current_sources = all_nodes_df.at[target_idx[0], "Source Nodes"]
            if isinstance(current_sources, list):  # 리스트 형태인 경우
                current_sources.append(source)
            else:  # 초기값이 없거나 비어 있을 경우
                current_sources = [source]
            # Source Nodes 갱신
            all_nodes_df.at[target_idx[0], "Source Nodes"] = current_sources

# Source Nodes를 문자열로 변환 (콤마로 연결)
all_nodes_df["Source Nodes"] = all_nodes_df["Source Nodes"].apply(
    lambda x: ", ".join(x) if isinstance(x, list) else ""
)

# 정렬: 중심 노드 기준, 빈도수 기준 내림차순
all_nodes_df = all_nodes_df.sort_values(by=["Node Type", "Frequency"], ascending=[True, False])

# 결과 저장
output_file = "network_nodes_data_oil_positive_negative.xlsx"
all_nodes_df.to_excel(output_file, index=False)

print(f"Processed node data saved to '{output_file}'")

[DEBUG] 모든 노드 데이터:
  Node ID Node Type   Label Sentiment  Frequency Source Nodes
0  않아서_긍정     연관 노드     않아서        긍정          5           []
1   오일리하지     중심 노드   오일리하지       무감정         34           []
2   않은_긍정     연관 노드      않은        긍정          1           []
3   세럼_긍정     연관 노드      세럼        긍정          3           []
4      오일     중심 노드      오일       무감정        278           []
5   있는_부정     연관 노드      있는        부정          2           []
6  오일덕분인지     중심 노드  오일덕분인지       무감정          3           []
7  성분이_긍정     연관 노드     성분이        긍정         13           []
8   섞여_긍정     연관 노드      섞여        긍정          6           []
9     오일이     중심 노드     오일이       무감정        451           []
[DEBUG] 모든 엣지 데이터:
Source: 오일리하지, Target: 않아서_긍정
Source: 오일리하지, Target: 않은_긍정
Source: 오일, Target: 세럼_긍정
Source: 오일덕분인지, Target: 있는_부정
Source: 오일, Target: 성분이_긍정
Source: 오일이, Target: 섞여_긍정
Source: 오일리하고, Target: 첨엔_부정
Source: 오일성분이, Target: 있어_긍정
Source: 오일층이, Target: 되고_긍정
Source: 성분을, Target: 결합하며

# 계절성

## 겨울, 가을
긍부정

In [85]:
from pyvis.network import Network
import pandas as pd
import re
from collections import Counter
from konlpy.tag import Okt

# 형태소 분석기 초기화
okt = Okt()

# 불용어 리스트 정의 (한국어 불용어)
with open(r"\Users\kyn03\OneDrive\바탕 화면\project_file\stopwords-ko(한국어불용어).txt", encoding='utf-8') as f:
    stopwords = f.read().splitlines()

# 추가 불용어 정의
stopwords += ['좀', '것', '달바', '미스트', '스프레이', '1으로', '1이라', '1에', '너무', '느낌', '잘', '그냥', '많이', 
              '있어서', '같아요', '스프레이가', '미스트가', '미스트는', '약간의', '생각보다', '있으며', '살짝', '같아서', 
              '느낌이', '저는', '진짜', '엄청', '합니다', '더', '안', '느낌은', '효과는', '조금', '분들은', '크림미스트라서',
              '크림미스트라', '크림', '크림이', '미스트라', '미스트라서', '그런지', '못', '이거', '완전', '아벤느', '다', '개', '걸',
              '매우','음','확','전','제','달바는','막','넘','너무너무','특히','근데','취향이','정말','덜','뭔가',
              '저의','굉장히','아니지만','아니고','해야할까요','제품','시',
              '있는데','샀는데','아니라','기분','썼을것같은데','바꿔봤는데','그런','사봤는데','해야할까요','저에게는',
              '하는데','아주','알겠더라구용','되게','나요','나서','나면서','진심','나니까','저한텐','저한테는','구매했어요',]

# 데이터 로드
data = pd.read_excel(r"\Users\kyn03\OneDrive\바탕 화면\project_file\sorted_merged_dalba.xlsx")
data = data.where(pd.notna(data), None)  # 결측치 처리
data = data.reset_index()

# 인덱스 넘버가 3 + 7 * n인 경우 필터링하여 'Aspect2 Mapped Labels' 행 필터링
filtered_data = data.iloc[[i for i in range(len(data)) if (i - 2) % 7 == 0]]

# 키워드 설정
aspect_keywords = ["겨울", "가을"]

# 텍스트 전처리 함수
def preprocess_text(text):
    """텍스트에서 특수문자 제거 및 불용어 필터링"""
    cleaned_text = re.sub(r"[^가-힣0-9\s]", "", text)
    tokens = [word for word in cleaned_text.split() if word not in stopwords]
    return tokens

# 네트워크 그래프 생성을 위한 데이터 수집
nodes_data = set()  # 중복 방지를 위해 set 사용
all_previous_tokens = []

# 키워드와 관련된 단어 연결
for idx, row in filtered_data.iterrows():
    original_idx = data[data['index'] == row['index']].index[0]
    tokenized_text_index = original_idx + 3  # 'Tokenized Text' 행
    sentiment_row_index = tokenized_text_index - 5  # 감정 데이터는 동일 열의 -5행에 위치

    if tokenized_text_index < len(data):

        for col_index in range(2, len(data.columns)):  # 각 열을 순회
            tokenized_text = str(data.iat[tokenized_text_index, col_index])

            # 감정 데이터 가져오기
            try:
                sentiment_value = str(data.iat[sentiment_row_index, col_index]).strip()
            except Exception as e:
                print(f"[ERROR] 감정 데이터를 가져오는 중 에러 발생: {e}")
                sentiment_value = None

            # Null 또는 빈 문자열 확인
            if not tokenized_text or tokenized_text.strip() == "":
                continue

            # 키워드 포함 여부 확인
            if not any(keyword in tokenized_text for keyword in aspect_keywords):
                continue

            # 키워드가 포함된 경우 처리
            print(f"[INFO] 키워드 포함 텍스트: {tokenized_text}, 감정: {sentiment_value}")

            # 기준 셀의 왼쪽과 오른쪽 셀 값을 추출
            left_text = (
                str(data.iat[tokenized_text_index, col_index - 1])
                if col_index > 2  # 왼쪽 셀이 존재하는 경우
                else None
            )
            right_text = (
                str(data.iat[tokenized_text_index, col_index + 1])
                if col_index < len(data.columns) - 1  # 오른쪽 셀이 존재하는 경우
                else None
            )

            # 제외할 접미사를 리스트로 관리
            EXCLUDED_SUFFIXES = ("-")
            EXCLUDED_KEYWORDS = ["-"]
            # 왼쪽 셀 추가
            if left_text and left_text.strip():
                stripped_left_text = left_text.strip()  # 공백 제거
                # 접미사 검사
                if any(stripped_left_text.endswith(suffix) for suffix in EXCLUDED_SUFFIXES):
                    print(f"[INFO] 왼쪽 셀이 '{EXCLUDED_SUFFIXES}' 중 하나로 끝나서 제외됨: {stripped_left_text}")

                elif any(keyword in stripped_left_text for keyword in EXCLUDED_KEYWORDS):
                    print(f"[INFO] 왼쪽 셀이 제외 키워드를 포함하여 제외됨: {stripped_left_text}")
                else:
                    nodes_data.add((tokenized_text, stripped_left_text, tokenized_text_index, col_index))  # set에 추가
                    all_previous_tokens.append(stripped_left_text)

            # 오른쪽 셀 추가
            if right_text and right_text.strip():
                stripped_right_text = right_text.strip()  # 공백 제거
                # 키워드 포함 여부 확인
                if any(keyword in stripped_right_text for keyword in EXCLUDED_KEYWORDS):
                    print(f"[INFO] 오른쪽 셀이 제외 키워드를 포함하여 제외됨: {stripped_right_text}")
                else:
                    nodes_data.add((tokenized_text, right_text.strip(),tokenized_text_index,col_index))  # set에 추가
                    all_previous_tokens.append(right_text.strip())

 

# 빈도수 카운터 생성
if not all_previous_tokens:
    print("[ERROR] 관련된 서술어가 없습니다. 전처리 또는 데이터 로드 로직을 확인하세요.")
else:
    # 감정별로 빈도수를 따로 저장하기 위한 딕셔너리
    sentiment_frequency = {
        "긍정": Counter(),
        "부정": Counter(),
        "중립": Counter()
    }

    # 전체 빈도수 저장
    total_frequency = Counter()

    # 빈도수 계산
    for tokenized_text, related_text, tokenized_text_index, col_index in nodes_data:
        # 감정 값 가져오기
        try:
            sentiment_value = str(data.iat[tokenized_text_index - 5, col_index]).strip()
            # B-긍정, I-긍정 같은 값을 "긍정"으로 통합
            if sentiment_value.startswith("B-") or sentiment_value.startswith("I-"):
                sentiment_value = sentiment_value[2:]
        except Exception as e:
            print(f"[ERROR] 감정 데이터를 가져오는 중 에러 발생: {e}")
            sentiment_value = None

        # 유효한 감정 값인지 확인
        if sentiment_value in sentiment_frequency:
            sentiment_frequency[sentiment_value][related_text] += 1
            total_frequency[related_text] += 1  # 전체 빈도수 업데이트



[INFO] 키워드 포함 텍스트: 가을과, 감정: O
[INFO] 키워드 포함 텍스트: 겨울에, 감정: O
[INFO] 키워드 포함 텍스트: 가을부터, 감정: B-긍정
[INFO] 키워드 포함 텍스트: 겨울, 감정: O
[INFO] 키워드 포함 텍스트: 겨울에, 감정: O
[INFO] 키워드 포함 텍스트: 겨울철, 감정: I-긍정
[INFO] 키워드 포함 텍스트: 겨울이, 감정: O
[INFO] 키워드 포함 텍스트: 가을, 감정: B-긍정
[INFO] 키워드 포함 텍스트: 겨울, 감정: I-긍정
[INFO] 키워드 포함 텍스트: 겨울, 감정: O
[INFO] 키워드 포함 텍스트: 겨울, 감정: O
[INFO] 키워드 포함 텍스트: 겨울엔, 감정: O
[INFO] 키워드 포함 텍스트: 겨울, 감정: I-긍정
[INFO] 키워드 포함 텍스트: 겨울의, 감정: O
[INFO] 키워드 포함 텍스트: 가을이나, 감정: B-긍정
[INFO] 키워드 포함 텍스트: 겨울철에, 감정: I-긍정
[INFO] 키워드 포함 텍스트: 가을이나, 감정: B-긍정
[INFO] 키워드 포함 텍스트: 겨울철에, 감정: I-긍정
[INFO] 키워드 포함 텍스트: 가을과, 감정: O
[INFO] 키워드 포함 텍스트: 겨울철에, 감정: O
[INFO] 키워드 포함 텍스트: 겨울까지, 감정: B-긍정
[INFO] 키워드 포함 텍스트: 겨울에는, 감정: O
[INFO] 키워드 포함 텍스트: 가을, 감정: B-긍정
[INFO] 키워드 포함 텍스트: 겨울에는, 감정: I-긍정
[INFO] 키워드 포함 텍스트: 겨울, 감정: O
[INFO] 키워드 포함 텍스트: 겨울에, 감정: B-긍정
[INFO] 키워드 포함 텍스트: 겨울철에는, 감정: I-긍정
[INFO] 키워드 포함 텍스트: 가을되면서, 감정: O
[INFO] 키워드 포함 텍스트: 가을, 감정: B-긍정
[INFO] 키워드 포함 텍스트: 겨울, 감정: I-긍정
[INFO] 키워드 포함 텍스트: 가을, 감정: B-긍정
[INFO] 키워드 포함 텍스트

In [86]:
# 네트워크 그래프 생성
net = Network(notebook=True, cdn_resources="in_line")
net.toggle_physics(True)

# 중심 노드와 연관 노드 간 연결 정보 기록
connected_nodes = {}
center_frequency = Counter()

# 그래프 노드 및 엣지 추가
EXCLUDED_MAIN_KEYWORDS = ["-"]

for main_token, related_token, sentiment_row_index, col_index in nodes_data:
    main_node_key = main_token.strip()  # 중심 노드 키 생성
    print(f"[DEBUG] Processing pair: main_token={main_token}, related_token={related_token}")

    # 유효성 검증 및 불용어 필터링
    if not re.match(r"^[가-힣]+$", main_node_key) or not re.match(r"^[가-힣]+$", related_token):
        print(f"[INFO] main_token 또는 related_token이 유효하지 않음: {main_token}, {related_token}")
        continue
    if main_token in stopwords or related_token in stopwords:
        print(f"[INFO] 불용어로 제외됨: {main_token} 또는 {related_token}")
        continue
    if any(keyword in main_node_key for keyword in EXCLUDED_MAIN_KEYWORDS):
        print(f"[INFO] 중심 노드가 제외 키워드를 포함하여 추가되지 않음: {main_node_key}")
        continue

    # 감정 값 가져오기
    related_token_sentiment = None
    try:
        related_token_sentiment = str(data.iat[sentiment_row_index - 5, col_index]).strip()
        if related_token_sentiment.startswith("B-") or related_token_sentiment.startswith("I-"):
            related_token_sentiment = related_token_sentiment[2:]  # 통합
    except Exception as e:
        print(f"[ERROR] related_token 감정 데이터 가져오기 실패: {e}")
        continue

    # 연관 노드 추가
    related_token_frequency = sentiment_frequency.get(related_token_sentiment, {}).get(related_token, 0)
    if related_token_frequency >= 1:
        if "중립" in related_token_sentiment or "O" in related_token_sentiment:
            print(f"[INFO] 연관 노드가 '중립' 또는 'O' 감정으로 추가되지 않음: {related_token}")
            continue

        related_node_key = f"{related_token}_{related_token_sentiment}"
        if related_node_key not in net.get_nodes():
            node_size = min(15 + related_token_frequency * 2, 50)
            related_token_color = {
                "긍정": "#6EC664",
                "중립": "#BDC3C7",
                "부정": "#E74C3C"
            }.get(related_token_sentiment, "black")
            net.add_node(
                related_node_key,
                label=related_token,
                color=related_token_color,
                size=node_size,
                title=f"{related_token} (빈도: {related_token_frequency})"
            )
            print(f"[DEBUG] 추가된 연관 노드: {related_node_key}")

        # 엣지 및 중심 노드 추가
        if main_node_key not in net.get_nodes():
            net.add_node(
                main_node_key,
                label=main_token,
                color={"background": "white", "border": "black"},
                size=25,
                title=f"{main_token} (빈도: 0)"  # 빈도수는 추후 갱신
            )
        # 엣지 추가
        net.add_edge(main_node_key, related_node_key, color=related_token_color)
        print(f"[DEBUG] 엣지 추가됨: {main_node_key} -> {related_node_key}")
        connected_nodes.setdefault(main_node_key, []).append(related_node_key)

# 중심 노드 빈도 계산 (화면에 표시된 연관 노드 기반)
center_frequency.clear()  # 기존 빈도수 초기화
for main_node_key, related_nodes in connected_nodes.items():
    for related_node_key in related_nodes:
        related_token = related_node_key.split("_")[0]
        related_token_sentiment = related_node_key.split("_")[1]
        related_token_frequency = sentiment_frequency.get(related_token_sentiment, {}).get(related_token, 0)
        center_frequency[main_node_key] += related_token_frequency
        print(f"[DEBUG] 중심 노드 '{main_node_key}'에 '{related_token}'의 빈도 {related_token_frequency} 추가됨")

# 중심 노드 빈도 업데이트 (화면에 표시된 값으로 갱신)
for node in net.nodes:
    if node["id"] in center_frequency:
        node["title"] = f"{node['id']} (빈도: {center_frequency[node['id']]})"
        print(f"[DEBUG] 중심 노드 '{node['id']}'의 빈도 갱신됨: {center_frequency[node['id']]}")

# 디버깅: 최종 중심 노드 빈도수 출력
print("[DEBUG] 최종 중심 노드 빈도수:")
for node, freq in center_frequency.items():
    print(f"{node}: {freq}")

# 네트워크 그래프 시각화
net.show("fall_winter_positive_negative.html")


[DEBUG] Processing pair: main_token=가을, related_token=,
[INFO] main_token 또는 related_token이 유효하지 않음: 가을, ,
[DEBUG] Processing pair: main_token=겨울, related_token=그리고
[INFO] 불용어로 제외됨: 겨울 또는 그리고
[DEBUG] Processing pair: main_token=겨울철이라, related_token=피부
[DEBUG] Processing pair: main_token=겨울에는, related_token=.
[INFO] main_token 또는 related_token이 유효하지 않음: 겨울에는, .
[DEBUG] Processing pair: main_token=겨울에, related_token=추운
[DEBUG] 추가된 연관 노드: 추운_긍정
[DEBUG] 엣지 추가됨: 겨울에 -> 추운_긍정
[DEBUG] Processing pair: main_token=겨울에도, related_token=재구매
[DEBUG] Processing pair: main_token=겨울철이라, related_token=수분미스트보다는
[DEBUG] Processing pair: main_token=겨울에, related_token=구매했어요
[INFO] 불용어로 제외됨: 겨울에 또는 구매했어요
[DEBUG] Processing pair: main_token=겨울에도, related_token=사용하기
[DEBUG] 추가된 연관 노드: 사용하기_긍정
[DEBUG] 엣지 추가됨: 겨울에도 -> 사용하기_긍정
[DEBUG] Processing pair: main_token=겨울철에, related_token=잘
[INFO] 불용어로 제외됨: 겨울철에 또는 잘
[DEBUG] Processing pair: main_token=겨울에, related_token=잘사용했어요
[DEBUG] 추가된 연관 노드: 잘사용했어요_긍정
[DEBUG] 엣지 추

In [87]:
import pandas as pd
import re

# 네트워크에 표시된 모든 노드 데이터 가져오기
all_nodes_data = []
for node in net.nodes:
    # 노드 정보 추출
    node_id = node["id"]
    node_label = node.get("label", "")
    node_color = node.get("color", "")
    node_title = node.get("title", "")  # 빈도수 정보 포함 가능

    # 노드 종류 결정
    if "_" in node_id:  # "_"이 포함된 경우 연관 노드
        node_type = "연관 노드"
    else:  # "_"이 없는 경우 중심 노드
        node_type = "중심 노드"

    # 감정 추출 (색상 기반)
    if node_color == "#6EC664":  # 연한 녹색
        node_sentiment = "긍정"
    elif node_color == "#BDC3C7":  # 연한 회색 (중립)
        node_sentiment = "중립"
    elif node_color == "#E74C3C":  # 연한 빨간색 (부정)
        node_sentiment = "부정"
    else:
        node_sentiment = "무감정"

    # 빈도수 추출 (Title에서 추출)
    try:
        node_frequency = int(re.search(r"\(빈도: (\d+)\)", node_title).group(1))
    except AttributeError:
        node_frequency = 0  # 빈도수 정보가 없으면 0으로 설정

    # 데이터 추가
    all_nodes_data.append({
        "Node ID": node_id,
        "Node Type": node_type,
        "Label": node_label,
        "Sentiment": node_sentiment,
        "Frequency": node_frequency,
        "Source Nodes": []  # 초기값으로 빈 리스트 설정 (연관 노드일 경우 추가적으로 설정)
    })

# DataFrame으로 변환
all_nodes_df = pd.DataFrame(all_nodes_data)

# 디버깅: 모든 노드 출력
print("[DEBUG] 모든 노드 데이터:")
print(all_nodes_df.head(10))

# 디버깅: 모든 엣지 데이터 출력
print("[DEBUG] 모든 엣지 데이터:")
for edge in net.edges:
    print(f"Source: {edge['from']}, Target: {edge['to']}")

# 엣지 데이터를 활용하여 Source Nodes 설정
for edge in net.edges:
    source = edge["from"]
    target = edge["to"]

    # 디버깅: 현재 처리 중인 엣지
    print(f"[DEBUG] 처리 중인 엣지: Source={source}, Target={target}")

    # 타겟 노드가 연관 노드인 경우 Source Nodes에 소스 노드 추가
    if "_" in target:  # 타겟 노드가 연관 노드
        target_idx = all_nodes_df[all_nodes_df["Node ID"] == target].index
        if not target_idx.empty:
            current_sources = all_nodes_df.at[target_idx[0], "Source Nodes"]
            if isinstance(current_sources, list):  # 리스트 형태인 경우
                current_sources.append(source)
            else:  # 초기값이 없거나 비어 있을 경우
                current_sources = [source]
            # Source Nodes 갱신
            all_nodes_df.at[target_idx[0], "Source Nodes"] = current_sources

# Source Nodes를 문자열로 변환 (콤마로 연결)
all_nodes_df["Source Nodes"] = all_nodes_df["Source Nodes"].apply(
    lambda x: ", ".join(x) if isinstance(x, list) else ""
)

# 정렬: 중심 노드 기준, 빈도수 기준 내림차순
all_nodes_df = all_nodes_df.sort_values(by=["Node Type", "Frequency"], ascending=[True, False])

# 결과 저장
output_file = "network_nodes_data_fall_winter_positive_negative.xlsx"
all_nodes_df.to_excel(output_file, index=False)

print(f"Processed node data saved to '{output_file}'")

[DEBUG] 모든 노드 데이터:
     Node ID Node Type   Label Sentiment  Frequency Source Nodes
0      추운_긍정     연관 노드      추운        긍정          1           []
1        겨울에     중심 노드     겨울에       무감정       1154           []
2    사용하기_긍정     연관 노드    사용하기        긍정         18           []
3       겨울에도     중심 노드    겨울에도       무감정         66           []
4  잘사용했어요_긍정     연관 노드  잘사용했어요        긍정          1           []
5      쓰기_긍정     연관 노드      쓰기        긍정         18           []
6       겨울철에     중심 노드    겨울철에       무감정        355           []
7    느낌이라_긍정     연관 노드    느낌이라        긍정          2           []
8         겨울     중심 노드      겨울       무감정        391           []
9   수부지인데_긍정     연관 노드   수부지인데        긍정          1           []
[DEBUG] 모든 엣지 데이터:
Source: 겨울에, Target: 추운_긍정
Source: 겨울에도, Target: 사용하기_긍정
Source: 겨울에, Target: 잘사용했어요_긍정
Source: 겨울에, Target: 쓰기_긍정
Source: 겨울철에, Target: 쓰기_긍정
Source: 겨울, Target: 느낌이라_긍정
Source: 겨울엔, Target: 수부지인데_긍정
Source: 겨울에, Target: 건조할때마다_긍정
Source: 겨울, Tar

## 여름
긍부정

In [88]:
from pyvis.network import Network
import pandas as pd
import re
from collections import Counter
from konlpy.tag import Okt

# 형태소 분석기 초기화
okt = Okt()

# 불용어 리스트 정의 (한국어 불용어)
with open(r"\Users\kyn03\OneDrive\바탕 화면\project_file\stopwords-ko(한국어불용어).txt", encoding='utf-8') as f:
    stopwords = f.read().splitlines()

# 추가 불용어 정의
stopwords += ['좀', '것', '달바', '미스트', '스프레이', '1으로', '1이라', '1에', '너무', '느낌', '잘', '그냥', '많이', 
              '있어서', '같아요', '스프레이가', '미스트가', '미스트는', '약간의', '생각보다', '있으며', '살짝', '같아서', 
              '느낌이', '저는', '진짜', '엄청', '합니다', '더', '안', '느낌은', '효과는', '조금', '분들은', '크림미스트라서',
              '크림미스트라', '크림', '크림이', '미스트라', '미스트라서', '그런지', '못', '이거', '완전', '아벤느', '다', '개', '걸',
              '매우','음','확','전','제','달바는','막','넘','너무너무','특히','근데','취향이','정말','덜','뭔가',
              '저의','굉장히','아니지만','아니고','해야할까요','제품','시',
              '있는데','샀는데','아니라','기분','썼을것같은데','바꿔봤는데','그런','사봤는데','해야할까요','저에게는',
              '하는데','아주','알겠더라구용','되게','나요','나서','나면서','진심','나니까','저한텐','저한테는','구매했어요',]

# 데이터 로드
data = pd.read_excel(r"\Users\kyn03\OneDrive\바탕 화면\project_file\sorted_merged_dalba.xlsx")
data = data.where(pd.notna(data), None)  # 결측치 처리
data = data.reset_index()

# 인덱스 넘버가 3 + 7 * n인 경우 필터링하여 'Aspect2 Mapped Labels' 행 필터링
filtered_data = data.iloc[[i for i in range(len(data)) if (i - 2) % 7 == 0]]

# 키워드 설정
aspect_keywords = ["여름"]

# 텍스트 전처리 함수
def preprocess_text(text):
    """텍스트에서 특수문자 제거 및 불용어 필터링"""
    cleaned_text = re.sub(r"[^가-힣0-9\s]", "", text)
    tokens = [word for word in cleaned_text.split() if word not in stopwords]
    return tokens

# 네트워크 그래프 생성을 위한 데이터 수집
nodes_data = set()  # 중복 방지를 위해 set 사용
all_previous_tokens = []

# 키워드와 관련된 단어 연결
for idx, row in filtered_data.iterrows():
    original_idx = data[data['index'] == row['index']].index[0]
    tokenized_text_index = original_idx + 3  # 'Tokenized Text' 행
    sentiment_row_index = tokenized_text_index - 5  # 감정 데이터는 동일 열의 -5행에 위치

    if tokenized_text_index < len(data):

        for col_index in range(2, len(data.columns)):  # 각 열을 순회
            tokenized_text = str(data.iat[tokenized_text_index, col_index])

            # 감정 데이터 가져오기
            try:
                sentiment_value = str(data.iat[sentiment_row_index, col_index]).strip()
            except Exception as e:
                print(f"[ERROR] 감정 데이터를 가져오는 중 에러 발생: {e}")
                sentiment_value = None

            # Null 또는 빈 문자열 확인
            if not tokenized_text or tokenized_text.strip() == "":
                continue

            # 키워드 포함 여부 확인
            if not any(keyword in tokenized_text for keyword in aspect_keywords):
                continue

            # 키워드가 포함된 경우 처리
            print(f"[INFO] 키워드 포함 텍스트: {tokenized_text}, 감정: {sentiment_value}")

            # 기준 셀의 왼쪽과 오른쪽 셀 값을 추출
            left_text = (
                str(data.iat[tokenized_text_index, col_index - 1])
                if col_index > 2  # 왼쪽 셀이 존재하는 경우
                else None
            )
            right_text = (
                str(data.iat[tokenized_text_index, col_index + 1])
                if col_index < len(data.columns) - 1  # 오른쪽 셀이 존재하는 경우
                else None
            )

            # 제외할 접미사를 리스트로 관리
            EXCLUDED_SUFFIXES = ("-")
            EXCLUDED_KEYWORDS = ["-"]
            # 왼쪽 셀 추가
            if left_text and left_text.strip():
                stripped_left_text = left_text.strip()  # 공백 제거
                # 접미사 검사
                if any(stripped_left_text.endswith(suffix) for suffix in EXCLUDED_SUFFIXES):
                    print(f"[INFO] 왼쪽 셀이 '{EXCLUDED_SUFFIXES}' 중 하나로 끝나서 제외됨: {stripped_left_text}")

                elif any(keyword in stripped_left_text for keyword in EXCLUDED_KEYWORDS):
                    print(f"[INFO] 왼쪽 셀이 제외 키워드를 포함하여 제외됨: {stripped_left_text}")
                else:
                    nodes_data.add((tokenized_text, stripped_left_text, tokenized_text_index, col_index))  # set에 추가
                    all_previous_tokens.append(stripped_left_text)

            # 오른쪽 셀 추가
            if right_text and right_text.strip():
                stripped_right_text = right_text.strip()  # 공백 제거
                # 키워드 포함 여부 확인
                if any(keyword in stripped_right_text for keyword in EXCLUDED_KEYWORDS):
                    print(f"[INFO] 오른쪽 셀이 제외 키워드를 포함하여 제외됨: {stripped_right_text}")
                else:
                    nodes_data.add((tokenized_text, right_text.strip(),tokenized_text_index,col_index))  # set에 추가
                    all_previous_tokens.append(right_text.strip())

 

# 빈도수 카운터 생성
if not all_previous_tokens:
    print("[ERROR] 관련된 서술어가 없습니다. 전처리 또는 데이터 로드 로직을 확인하세요.")
else:
    # 감정별로 빈도수를 따로 저장하기 위한 딕셔너리
    sentiment_frequency = {
        "긍정": Counter(),
        "부정": Counter(),
        "중립": Counter()
    }

    # 전체 빈도수 저장
    total_frequency = Counter()

    # 빈도수 계산
    for tokenized_text, related_text, tokenized_text_index, col_index in nodes_data:
        # 감정 값 가져오기
        try:
            sentiment_value = str(data.iat[tokenized_text_index - 5, col_index]).strip()
            # B-긍정, I-긍정 같은 값을 "긍정"으로 통합
            if sentiment_value.startswith("B-") or sentiment_value.startswith("I-"):
                sentiment_value = sentiment_value[2:]
        except Exception as e:
            print(f"[ERROR] 감정 데이터를 가져오는 중 에러 발생: {e}")
            sentiment_value = None

        # 유효한 감정 값인지 확인
        if sentiment_value in sentiment_frequency:
            sentiment_frequency[sentiment_value][related_text] += 1
            total_frequency[related_text] += 1  # 전체 빈도수 업데이트




[INFO] 키워드 포함 텍스트: 여름에, 감정: O
[INFO] 키워드 포함 텍스트: 여름에, 감정: O
[INFO] 키워드 포함 텍스트: 여름이, 감정: O
[INFO] 키워드 포함 텍스트: 여름이, 감정: B-긍정
[INFO] 키워드 포함 텍스트: 여름이라, 감정: I-부정
[INFO] 키워드 포함 텍스트: 여름에, 감정: I-긍정
[INFO] 키워드 포함 텍스트: 여름에, 감정: B-긍정
[INFO] 키워드 포함 텍스트: 여름엔, 감정: O
[INFO] 키워드 포함 텍스트: 여름, 감정: B-긍정
[INFO] 키워드 포함 텍스트: 여름에도, 감정: I-긍정
[INFO] 키워드 포함 텍스트: 여름에도, 감정: O
[INFO] 키워드 포함 텍스트: 여름철에는, 감정: O
[INFO] 키워드 포함 텍스트: 여름에, 감정: B-긍정
[INFO] 키워드 포함 텍스트: 여름, 감정: I-긍정
[INFO] 키워드 포함 텍스트: 여름에, 감정: B-부정
[INFO] 키워드 포함 텍스트: 여름에, 감정: O
[INFO] 키워드 포함 텍스트: 여름엔, 감정: O
[INFO] 키워드 포함 텍스트: 여름에, 감정: B-부정
[INFO] 키워드 포함 텍스트: 여름에는, 감정: B-긍정
[INFO] 키워드 포함 텍스트: 여름철이여서, 감정: O
[INFO] 키워드 포함 텍스트: 여름에나, 감정: B-부정
[INFO] 키워드 포함 텍스트: 여름이니까, 감정: O
[INFO] 키워드 포함 텍스트: 여름, 감정: B-긍정
[INFO] 키워드 포함 텍스트: 여름, 감정: B-긍정
[INFO] 키워드 포함 텍스트: 여름엔, 감정: B-중립
[INFO] 키워드 포함 텍스트: 저는여름철에도, 감정: I-긍정
[INFO] 키워드 포함 텍스트: 여름철보다, 감정: B-중립
[INFO] 키워드 포함 텍스트: 저는여름철에도, 감정: I-긍정
[INFO] 키워드 포함 텍스트: 여름철보다, 감정: B-중립
[INFO] 키워드 포함 텍스트: 여름에, 감정: O
[INFO] 키워드 포함 텍스트: 여름에도

In [89]:
# 네트워크 그래프 생성
net = Network(notebook=True, cdn_resources="in_line")
net.toggle_physics(True)

# 중심 노드와 연관 노드 간 연결 정보 기록
connected_nodes = {}
center_frequency = Counter()

# 그래프 노드 및 엣지 추가
EXCLUDED_MAIN_KEYWORDS = ["-"]

for main_token, related_token, sentiment_row_index, col_index in nodes_data:
    main_node_key = main_token.strip()  # 중심 노드 키 생성
    print(f"[DEBUG] Processing pair: main_token={main_token}, related_token={related_token}")

    # 유효성 검증 및 불용어 필터링
    if not re.match(r"^[가-힣]+$", main_node_key) or not re.match(r"^[가-힣]+$", related_token):
        print(f"[INFO] main_token 또는 related_token이 유효하지 않음: {main_token}, {related_token}")
        continue
    if main_token in stopwords or related_token in stopwords:
        print(f"[INFO] 불용어로 제외됨: {main_token} 또는 {related_token}")
        continue
    if any(keyword in main_node_key for keyword in EXCLUDED_MAIN_KEYWORDS):
        print(f"[INFO] 중심 노드가 제외 키워드를 포함하여 추가되지 않음: {main_node_key}")
        continue

    # 감정 값 가져오기
    related_token_sentiment = None
    try:
        related_token_sentiment = str(data.iat[sentiment_row_index - 5, col_index]).strip()
        if related_token_sentiment.startswith("B-") or related_token_sentiment.startswith("I-"):
            related_token_sentiment = related_token_sentiment[2:]  # 통합
    except Exception as e:
        print(f"[ERROR] related_token 감정 데이터 가져오기 실패: {e}")
        continue

    # 연관 노드 추가
    related_token_frequency = sentiment_frequency.get(related_token_sentiment, {}).get(related_token, 0)
    if related_token_frequency >= 1:
        if "중립" in related_token_sentiment or "O" in related_token_sentiment:
            print(f"[INFO] 연관 노드가 '중립' 또는 'O' 감정으로 추가되지 않음: {related_token}")
            continue

        related_node_key = f"{related_token}_{related_token_sentiment}"
        if related_node_key not in net.get_nodes():
            node_size = min(15 + related_token_frequency * 2, 50)
            related_token_color = {
                "긍정": "#6EC664",
                "중립": "#BDC3C7",
                "부정": "#E74C3C"
            }.get(related_token_sentiment, "black")
            net.add_node(
                related_node_key,
                label=related_token,
                color=related_token_color,
                size=node_size,
                title=f"{related_token} (빈도: {related_token_frequency})"
            )
            print(f"[DEBUG] 추가된 연관 노드: {related_node_key}")

        # 엣지 및 중심 노드 추가
        if main_node_key not in net.get_nodes():
            net.add_node(
                main_node_key,
                label=main_token,
                color={"background": "white", "border": "black"},
                size=25,
                title=f"{main_token} (빈도: 0)"  # 빈도수는 추후 갱신
            )
        # 엣지 추가
        net.add_edge(main_node_key, related_node_key, color=related_token_color)
        print(f"[DEBUG] 엣지 추가됨: {main_node_key} -> {related_node_key}")
        connected_nodes.setdefault(main_node_key, []).append(related_node_key)

# 중심 노드 빈도 계산 (화면에 표시된 연관 노드 기반)
center_frequency.clear()  # 기존 빈도수 초기화
for main_node_key, related_nodes in connected_nodes.items():
    for related_node_key in related_nodes:
        related_token = related_node_key.split("_")[0]
        related_token_sentiment = related_node_key.split("_")[1]
        related_token_frequency = sentiment_frequency.get(related_token_sentiment, {}).get(related_token, 0)
        center_frequency[main_node_key] += related_token_frequency
        print(f"[DEBUG] 중심 노드 '{main_node_key}'에 '{related_token}'의 빈도 {related_token_frequency} 추가됨")

# 중심 노드 빈도 업데이트 (화면에 표시된 값으로 갱신)
for node in net.nodes:
    if node["id"] in center_frequency:
        node["title"] = f"{node['id']} (빈도: {center_frequency[node['id']]})"
        print(f"[DEBUG] 중심 노드 '{node['id']}'의 빈도 갱신됨: {center_frequency[node['id']]}")

# 디버깅: 최종 중심 노드 빈도수 출력
print("[DEBUG] 최종 중심 노드 빈도수:")
for node, freq in center_frequency.items():
    print(f"{node}: {freq}")

# 네트워크 그래프 시각화
net.show("summer_positive_negative.html")


[DEBUG] Processing pair: main_token=여름에는, related_token=스타일
[DEBUG] Processing pair: main_token=여름에도, related_token=쓰시고
[DEBUG] 추가된 연관 노드: 쓰시고_긍정
[DEBUG] 엣지 추가됨: 여름에도 -> 쓰시고_긍정
[DEBUG] Processing pair: main_token=여름을, related_token=제외하고는
[DEBUG] 추가된 연관 노드: 제외하고는_긍정
[DEBUG] 엣지 추가됨: 여름을 -> 제외하고는_긍정
[DEBUG] Processing pair: main_token=여름에, related_token=사용하기에는
[DEBUG] Processing pair: main_token=여름에는, related_token=없고
[DEBUG] 추가된 연관 노드: 없고_긍정
[DEBUG] 엣지 추가됨: 여름에는 -> 없고_긍정
[DEBUG] Processing pair: main_token=여름에, related_token=가볍게
[DEBUG] 추가된 연관 노드: 가볍게_긍정
[DEBUG] 엣지 추가됨: 여름에 -> 가볍게_긍정
[DEBUG] Processing pair: main_token=여름엔, related_token=있어
[DEBUG] 추가된 연관 노드: 있어_부정
[DEBUG] 엣지 추가됨: 여름엔 -> 있어_부정
[DEBUG] Processing pair: main_token=여름에는, related_token=엄청
[INFO] 불용어로 제외됨: 여름에는 또는 엄청
[DEBUG] Processing pair: main_token=여름에, related_token=끈적임이없어서
[DEBUG] 추가된 연관 노드: 끈적임이없어서_긍정
[DEBUG] 엣지 추가됨: 여름에 -> 끈적임이없어서_긍정
[DEBUG] Processing pair: main_token=저는여름철에도, related_token=좋습니다👍👍
[INFO] main_token 또

In [90]:
import pandas as pd
import re

# 네트워크에 표시된 모든 노드 데이터 가져오기
all_nodes_data = []
for node in net.nodes:
    # 노드 정보 추출
    node_id = node["id"]
    node_label = node.get("label", "")
    node_color = node.get("color", "")
    node_title = node.get("title", "")  # 빈도수 정보 포함 가능

    # 노드 종류 결정
    if "_" in node_id:  # "_"이 포함된 경우 연관 노드
        node_type = "연관 노드"
    else:  # "_"이 없는 경우 중심 노드
        node_type = "중심 노드"

    # 감정 추출 (색상 기반)
    if node_color == "#6EC664":  # 연한 녹색
        node_sentiment = "긍정"
    elif node_color == "#BDC3C7":  # 연한 회색 (중립)
        node_sentiment = "중립"
    elif node_color == "#E74C3C":  # 연한 빨간색 (부정)
        node_sentiment = "부정"
    else:
        node_sentiment = "무감정"

    # 빈도수 추출 (Title에서 추출)
    try:
        node_frequency = int(re.search(r"\(빈도: (\d+)\)", node_title).group(1))
    except AttributeError:
        node_frequency = 0  # 빈도수 정보가 없으면 0으로 설정

    # 데이터 추가
    all_nodes_data.append({
        "Node ID": node_id,
        "Node Type": node_type,
        "Label": node_label,
        "Sentiment": node_sentiment,
        "Frequency": node_frequency,
        "Source Nodes": []  # 초기값으로 빈 리스트 설정 (연관 노드일 경우 추가적으로 설정)
    })

# DataFrame으로 변환
all_nodes_df = pd.DataFrame(all_nodes_data)

# 디버깅: 모든 노드 출력
print("[DEBUG] 모든 노드 데이터:")
print(all_nodes_df.head(10))

# 디버깅: 모든 엣지 데이터 출력
print("[DEBUG] 모든 엣지 데이터:")
for edge in net.edges:
    print(f"Source: {edge['from']}, Target: {edge['to']}")

# 엣지 데이터를 활용하여 Source Nodes 설정
for edge in net.edges:
    source = edge["from"]
    target = edge["to"]

    # 디버깅: 현재 처리 중인 엣지
    print(f"[DEBUG] 처리 중인 엣지: Source={source}, Target={target}")

    # 타겟 노드가 연관 노드인 경우 Source Nodes에 소스 노드 추가
    if "_" in target:  # 타겟 노드가 연관 노드
        target_idx = all_nodes_df[all_nodes_df["Node ID"] == target].index
        if not target_idx.empty:
            current_sources = all_nodes_df.at[target_idx[0], "Source Nodes"]
            if isinstance(current_sources, list):  # 리스트 형태인 경우
                current_sources.append(source)
            else:  # 초기값이 없거나 비어 있을 경우
                current_sources = [source]
            # Source Nodes 갱신
            all_nodes_df.at[target_idx[0], "Source Nodes"] = current_sources

# Source Nodes를 문자열로 변환 (콤마로 연결)
all_nodes_df["Source Nodes"] = all_nodes_df["Source Nodes"].apply(
    lambda x: ", ".join(x) if isinstance(x, list) else ""
)

# 정렬: 중심 노드 기준, 빈도수 기준 내림차순
all_nodes_df = all_nodes_df.sort_values(by=["Node Type", "Frequency"], ascending=[True, False])

# 결과 저장
output_file = "network_nodes_data_summer_positive_negative.xlsx"
all_nodes_df.to_excel(output_file, index=False)

print(f"Processed node data saved to '{output_file}'")

[DEBUG] 모든 노드 데이터:
    Node ID Node Type  Label Sentiment  Frequency Source Nodes
0    쓰시고_긍정     연관 노드    쓰시고        긍정          1           []
1      여름에도     중심 노드   여름에도       무감정         39           []
2  제외하고는_긍정     연관 노드  제외하고는        긍정          1           []
3       여름을     중심 노드    여름을       무감정          1           []
4     없고_긍정     연관 노드     없고        긍정          1           []
5      여름에는     중심 노드   여름에는       무감정         24           []
6    가볍게_긍정     연관 노드    가볍게        긍정          2           []
7       여름에     중심 노드    여름에       무감정         90           []
8     있어_부정     연관 노드     있어        부정          1           []
9       여름엔     중심 노드    여름엔       무감정         30           []
[DEBUG] 모든 엣지 데이터:
Source: 여름에도, Target: 쓰시고_긍정
Source: 여름을, Target: 제외하고는_긍정
Source: 여름에는, Target: 없고_긍정
Source: 여름에, Target: 가볍게_긍정
Source: 여름엔, Target: 있어_부정
Source: 여름에, Target: 끈적임이없어서_긍정
Source: 여름보다는, Target: 확실히_긍정
Source: 여름에, Target: 특히나_긍정
Source: 여름에나, Target: 뿌리는_부정
Source: 

# 피부타입

## 건성

In [91]:
from pyvis.network import Network
import pandas as pd
import re
from collections import Counter
from konlpy.tag import Okt

# 형태소 분석기 초기화
okt = Okt()

# 불용어 리스트 정의 (한국어 불용어)
with open(r"\Users\kyn03\OneDrive\바탕 화면\project_file\stopwords-ko(한국어불용어).txt", encoding='utf-8') as f:
    stopwords = f.read().splitlines()

# 추가 불용어 정의
stopwords += ['좀', '것', '달바', '미스트', '스프레이', '1으로', '1이라', '1에', '너무', '느낌', '잘', '그냥', '많이', 
              '있어서', '같아요', '스프레이가', '미스트가', '미스트는', '약간의', '생각보다', '있으며', '살짝', '같아서', 
              '느낌이', '저는', '진짜', '엄청', '합니다', '더', '안', '느낌은', '효과는', '조금', '분들은', '크림미스트라서',
              '크림미스트라', '크림', '크림이', '미스트라', '미스트라서', '그런지', '못', '이거', '완전', '아벤느', '다', '개', '걸',
              '매우','음','확','전','제','달바는','막','넘','너무너무','특히','근데','취향이','정말','덜','뭔가',
              '저의','굉장히','아니지만','아니고','해야할까요','제품','시',
              '있는데','샀는데','아니라','기분','썼을것같은데','바꿔봤는데','그런','사봤는데','해야할까요','저에게는',
              '하는데','아주','알겠더라구용','되게','나요','나서','나면서','진심','나니까','저한텐','저한테는','구매했어요',]

# 데이터 로드
data = pd.read_excel(r"\Users\kyn03\OneDrive\바탕 화면\project_file\sorted_merged_dalba.xlsx")
data = data.where(pd.notna(data), None)  # 결측치 처리
data = data.reset_index()

# 인덱스 넘버가 3 + 7 * n인 경우 필터링하여 'Aspect2 Mapped Labels' 행 필터링
filtered_data = data.iloc[[i for i in range(len(data)) if (i - 2) % 7 == 0]]

# 키워드 설정
aspect_keywords = ["건성","수부지"]

# 텍스트 전처리 함수
def preprocess_text(text):
    """텍스트에서 특수문자 제거 및 불용어 필터링"""
    cleaned_text = re.sub(r"[^가-힣0-9\s]", "", text)
    tokens = [word for word in cleaned_text.split() if word not in stopwords]
    return tokens

# 네트워크 그래프 생성을 위한 데이터 수집
nodes_data = set()  # 중복 방지를 위해 set 사용
all_previous_tokens = []

# 키워드와 관련된 단어 연결
for idx, row in filtered_data.iterrows():
    original_idx = data[data['index'] == row['index']].index[0]
    tokenized_text_index = original_idx + 3  # 'Tokenized Text' 행
    sentiment_row_index = tokenized_text_index - 5  # 감정 데이터는 동일 열의 -5행에 위치

    if tokenized_text_index < len(data):

        for col_index in range(2, len(data.columns)):  # 각 열을 순회
            tokenized_text = str(data.iat[tokenized_text_index, col_index])

            # 감정 데이터 가져오기
            try:
                sentiment_value = str(data.iat[sentiment_row_index, col_index]).strip()
            except Exception as e:
                print(f"[ERROR] 감정 데이터를 가져오는 중 에러 발생: {e}")
                sentiment_value = None

            # Null 또는 빈 문자열 확인
            if not tokenized_text or tokenized_text.strip() == "":
                continue

            # 키워드 포함 여부 확인
            if not any(keyword in tokenized_text for keyword in aspect_keywords):
                continue

            # 키워드가 포함된 경우 처리
            print(f"[INFO] 키워드 포함 텍스트: {tokenized_text}, 감정: {sentiment_value}")

            # 기준 셀의 왼쪽과 오른쪽 셀 값을 추출
            left_text = (
                str(data.iat[tokenized_text_index, col_index - 1])
                if col_index > 2  # 왼쪽 셀이 존재하는 경우
                else None
            )
            right_text = (
                str(data.iat[tokenized_text_index, col_index + 1])
                if col_index < len(data.columns) - 1  # 오른쪽 셀이 존재하는 경우
                else None
            )

            # 제외할 접미사를 리스트로 관리
            EXCLUDED_SUFFIXES = ("-")
            EXCLUDED_KEYWORDS = ["-"]
            # 왼쪽 셀 추가
            if left_text and left_text.strip():
                stripped_left_text = left_text.strip()  # 공백 제거
                # 접미사 검사
                if any(stripped_left_text.endswith(suffix) for suffix in EXCLUDED_SUFFIXES):
                    print(f"[INFO] 왼쪽 셀이 '{EXCLUDED_SUFFIXES}' 중 하나로 끝나서 제외됨: {stripped_left_text}")

                elif any(keyword in stripped_left_text for keyword in EXCLUDED_KEYWORDS):
                    print(f"[INFO] 왼쪽 셀이 제외 키워드를 포함하여 제외됨: {stripped_left_text}")
                else:
                    nodes_data.add((tokenized_text, stripped_left_text, tokenized_text_index, col_index))  # set에 추가
                    all_previous_tokens.append(stripped_left_text)

            # 오른쪽 셀 추가
            if right_text and right_text.strip():
                stripped_right_text = right_text.strip()  # 공백 제거
                # 키워드 포함 여부 확인
                if any(keyword in stripped_right_text for keyword in EXCLUDED_KEYWORDS):
                    print(f"[INFO] 오른쪽 셀이 제외 키워드를 포함하여 제외됨: {stripped_right_text}")
                else:
                    nodes_data.add((tokenized_text, right_text.strip(),tokenized_text_index,col_index))  # set에 추가
                    all_previous_tokens.append(right_text.strip())

 

# 빈도수 카운터 생성
if not all_previous_tokens:
    print("[ERROR] 관련된 서술어가 없습니다. 전처리 또는 데이터 로드 로직을 확인하세요.")
else:
    # 감정별로 빈도수를 따로 저장하기 위한 딕셔너리
    sentiment_frequency = {
        "긍정": Counter(),
        "부정": Counter(),
        "중립": Counter()
    }

    # 전체 빈도수 저장
    total_frequency = Counter()

    # 빈도수 계산
    for tokenized_text, related_text, tokenized_text_index, col_index in nodes_data:
        # 감정 값 가져오기
        try:
            sentiment_value = str(data.iat[tokenized_text_index - 5, col_index]).strip()
            # B-긍정, I-긍정 같은 값을 "긍정"으로 통합
            if sentiment_value.startswith("B-") or sentiment_value.startswith("I-"):
                sentiment_value = sentiment_value[2:]
        except Exception as e:
            print(f"[ERROR] 감정 데이터를 가져오는 중 에러 발생: {e}")
            sentiment_value = None

        # 유효한 감정 값인지 확인
        if sentiment_value in sentiment_frequency:
            sentiment_frequency[sentiment_value][related_text] += 1
            total_frequency[related_text] += 1  # 전체 빈도수 업데이트




[INFO] 키워드 포함 텍스트: 건성, 감정: O
[INFO] 키워드 포함 텍스트: 건성, 감정: O
[INFO] 키워드 포함 텍스트: 건성, 감정: O
[INFO] 키워드 포함 텍스트: 건성, 감정: O
[INFO] 키워드 포함 텍스트: 건성, 감정: O
[INFO] 키워드 포함 텍스트: 건성, 감정: O
[INFO] 키워드 포함 텍스트: 건성, 감정: O
[INFO] 키워드 포함 텍스트: 건성, 감정: B-긍정
[INFO] 키워드 포함 텍스트: 건성이라, 감정: O
[INFO] 키워드 포함 텍스트: 수부지, 감정: I-부정
[INFO] 키워드 포함 텍스트: 건성인데, 감정: B-긍정
[INFO] 키워드 포함 텍스트: 건성, 감정: B-긍정
[INFO] 키워드 포함 텍스트: 건성인데, 감정: B-긍정
[INFO] 키워드 포함 텍스트: 건성, 감정: B-긍정
[INFO] 키워드 포함 텍스트: 건성이라, 감정: B-긍정
[INFO] 키워드 포함 텍스트: 건성, 감정: O
[INFO] 키워드 포함 텍스트: 건성이라, 감정: O
[INFO] 키워드 포함 텍스트: 건성인데, 감정: B-긍정
[INFO] 키워드 포함 텍스트: 건성이, 감정: I-긍정
[INFO] 키워드 포함 텍스트: 속건성도, 감정: I-긍정
[INFO] 키워드 포함 텍스트: 건성피부, 감정: B-긍정
[INFO] 키워드 포함 텍스트: 건성이든, 감정: B-긍정
[INFO] 키워드 포함 텍스트: 수부지라, 감정: B-긍정
[INFO] 키워드 포함 텍스트: 건성피부에, 감정: I-긍정
[INFO] 키워드 포함 텍스트: 건성피부에게, 감정: B-긍정
[INFO] 키워드 포함 텍스트: 수부지라서, 감정: O
[INFO] 키워드 포함 텍스트: 건성은, 감정: B-부정
[INFO] 키워드 포함 텍스트: 건성이신분들는, 감정: O
[INFO] 키워드 포함 텍스트: 악건성인, 감정: O
[INFO] 키워드 포함 텍스트: 수부지, 감정: O
[INFO] 키워드 포함 텍스트: 악건성은, 감정: B-긍정
[INFO] 

In [92]:
# 네트워크 그래프 생성
net = Network(notebook=True, cdn_resources="in_line")
net.toggle_physics(True)

# 중심 노드와 연관 노드 간 연결 정보 기록
connected_nodes = {}
center_frequency = Counter()

# 그래프 노드 및 엣지 추가
EXCLUDED_MAIN_KEYWORDS = ["-"]

for main_token, related_token, sentiment_row_index, col_index in nodes_data:
    main_node_key = main_token.strip()  # 중심 노드 키 생성
    print(f"[DEBUG] Processing pair: main_token={main_token}, related_token={related_token}")

    # 유효성 검증 및 불용어 필터링
    if not re.match(r"^[가-힣]+$", main_node_key) or not re.match(r"^[가-힣]+$", related_token):
        print(f"[INFO] main_token 또는 related_token이 유효하지 않음: {main_token}, {related_token}")
        continue
    if main_token in stopwords or related_token in stopwords:
        print(f"[INFO] 불용어로 제외됨: {main_token} 또는 {related_token}")
        continue
    if any(keyword in main_node_key for keyword in EXCLUDED_MAIN_KEYWORDS):
        print(f"[INFO] 중심 노드가 제외 키워드를 포함하여 추가되지 않음: {main_node_key}")
        continue

    # 감정 값 가져오기
    related_token_sentiment = None
    try:
        related_token_sentiment = str(data.iat[sentiment_row_index - 5, col_index]).strip()
        if related_token_sentiment.startswith("B-") or related_token_sentiment.startswith("I-"):
            related_token_sentiment = related_token_sentiment[2:]  # 통합
    except Exception as e:
        print(f"[ERROR] related_token 감정 데이터 가져오기 실패: {e}")
        continue

    # 연관 노드 추가
    related_token_frequency = sentiment_frequency.get(related_token_sentiment, {}).get(related_token, 0)
    if related_token_frequency >= 1:
        if "중립" in related_token_sentiment or "O" in related_token_sentiment:
            print(f"[INFO] 연관 노드가 '중립' 또는 'O' 감정으로 추가되지 않음: {related_token}")
            continue

        related_node_key = f"{related_token}_{related_token_sentiment}"
        if related_node_key not in net.get_nodes():
            node_size = min(15 + related_token_frequency * 2, 50)
            related_token_color = {
                "긍정": "#6EC664",
                "중립": "#BDC3C7",
                "부정": "#E74C3C"
            }.get(related_token_sentiment, "black")
            net.add_node(
                related_node_key,
                label=related_token,
                color=related_token_color,
                size=node_size,
                title=f"{related_token} (빈도: {related_token_frequency})"
            )
            print(f"[DEBUG] 추가된 연관 노드: {related_node_key}")

        # 엣지 및 중심 노드 추가
        if main_node_key not in net.get_nodes():
            net.add_node(
                main_node_key,
                label=main_token,
                color={"background": "white", "border": "black"},
                size=25,
                title=f"{main_token} (빈도: 0)"  # 빈도수는 추후 갱신
            )
        # 엣지 추가
        net.add_edge(main_node_key, related_node_key, color=related_token_color)
        print(f"[DEBUG] 엣지 추가됨: {main_node_key} -> {related_node_key}")
        connected_nodes.setdefault(main_node_key, []).append(related_node_key)

# 중심 노드 빈도 계산 (화면에 표시된 연관 노드 기반)
center_frequency.clear()  # 기존 빈도수 초기화
for main_node_key, related_nodes in connected_nodes.items():
    for related_node_key in related_nodes:
        related_token = related_node_key.split("_")[0]
        related_token_sentiment = related_node_key.split("_")[1]
        related_token_frequency = sentiment_frequency.get(related_token_sentiment, {}).get(related_token, 0)
        center_frequency[main_node_key] += related_token_frequency
        print(f"[DEBUG] 중심 노드 '{main_node_key}'에 '{related_token}'의 빈도 {related_token_frequency} 추가됨")

# 중심 노드 빈도 업데이트 (화면에 표시된 값으로 갱신)
for node in net.nodes:
    if node["id"] in center_frequency:
        node["title"] = f"{node['id']} (빈도: {center_frequency[node['id']]})"
        print(f"[DEBUG] 중심 노드 '{node['id']}'의 빈도 갱신됨: {center_frequency[node['id']]}")

# 디버깅: 최종 중심 노드 빈도수 출력
print("[DEBUG] 최종 중심 노드 빈도수:")
for node, freq in center_frequency.items():
    print(f"{node}: {freq}")

# 네트워크 그래프 시각화
net.show("dry_positive_negative.html")


[DEBUG] Processing pair: main_token=건성인, related_token=써봤는데
[DEBUG] Processing pair: main_token=건성피부분들, related_token=~
[INFO] main_token 또는 related_token이 유효하지 않음: 건성피부분들, ~
[DEBUG] Processing pair: main_token=건성은, related_token=만족합니다
[DEBUG] 추가된 연관 노드: 만족합니다_긍정
[DEBUG] 엣지 추가됨: 건성은 -> 만족합니다_긍정
[DEBUG] Processing pair: main_token=악건성이라, related_token=피부가
[DEBUG] 추가된 연관 노드: 피부가_긍정
[DEBUG] 엣지 추가됨: 악건성이라 -> 피부가_긍정
[DEBUG] Processing pair: main_token=건성, related_token=피부라서
[DEBUG] 추가된 연관 노드: 피부라서_긍정
[DEBUG] 엣지 추가됨: 건성 -> 피부라서_긍정
[DEBUG] Processing pair: main_token=건성이라, related_token=워낙
[DEBUG] 추가된 연관 노드: 워낙_긍정
[DEBUG] 엣지 추가됨: 건성이라 -> 워낙_긍정
[DEBUG] Processing pair: main_token=극건성, related_token=,
[INFO] main_token 또는 related_token이 유효하지 않음: 극건성, ,
[DEBUG] Processing pair: main_token=건성이고, related_token=예민한
[DEBUG] Processing pair: main_token=건성이라, related_token=어머니는
[DEBUG] 추가된 연관 노드: 어머니는_긍정
[DEBUG] 엣지 추가됨: 건성이라 -> 어머니는_긍정
[DEBUG] Processing pair: main_token=건성이, related_token=되지만
[DEBUG]

In [93]:
import pandas as pd
import re

# 네트워크에 표시된 모든 노드 데이터 가져오기
all_nodes_data = []
for node in net.nodes:
    # 노드 정보 추출
    node_id = node["id"]
    node_label = node.get("label", "")
    node_color = node.get("color", "")
    node_title = node.get("title", "")  # 빈도수 정보 포함 가능

    # 노드 종류 결정
    if "_" in node_id:  # "_"이 포함된 경우 연관 노드
        node_type = "연관 노드"
    else:  # "_"이 없는 경우 중심 노드
        node_type = "중심 노드"

    # 감정 추출 (색상 기반)
    if node_color == "#6EC664":  # 연한 녹색
        node_sentiment = "긍정"
    elif node_color == "#BDC3C7":  # 연한 회색 (중립)
        node_sentiment = "중립"
    elif node_color == "#E74C3C":  # 연한 빨간색 (부정)
        node_sentiment = "부정"
    else:
        node_sentiment = "무감정"

    # 빈도수 추출 (Title에서 추출)
    try:
        node_frequency = int(re.search(r"\(빈도: (\d+)\)", node_title).group(1))
    except AttributeError:
        node_frequency = 0  # 빈도수 정보가 없으면 0으로 설정

    # 데이터 추가
    all_nodes_data.append({
        "Node ID": node_id,
        "Node Type": node_type,
        "Label": node_label,
        "Sentiment": node_sentiment,
        "Frequency": node_frequency,
        "Source Nodes": []  # 초기값으로 빈 리스트 설정 (연관 노드일 경우 추가적으로 설정)
    })

# DataFrame으로 변환
all_nodes_df = pd.DataFrame(all_nodes_data)

# 디버깅: 모든 노드 출력
print("[DEBUG] 모든 노드 데이터:")
print(all_nodes_df.head(10))

# 디버깅: 모든 엣지 데이터 출력
print("[DEBUG] 모든 엣지 데이터:")
for edge in net.edges:
    print(f"Source: {edge['from']}, Target: {edge['to']}")

# 엣지 데이터를 활용하여 Source Nodes 설정
for edge in net.edges:
    source = edge["from"]
    target = edge["to"]

    # 디버깅: 현재 처리 중인 엣지
    print(f"[DEBUG] 처리 중인 엣지: Source={source}, Target={target}")

    # 타겟 노드가 연관 노드인 경우 Source Nodes에 소스 노드 추가
    if "_" in target:  # 타겟 노드가 연관 노드
        target_idx = all_nodes_df[all_nodes_df["Node ID"] == target].index
        if not target_idx.empty:
            current_sources = all_nodes_df.at[target_idx[0], "Source Nodes"]
            if isinstance(current_sources, list):  # 리스트 형태인 경우
                current_sources.append(source)
            else:  # 초기값이 없거나 비어 있을 경우
                current_sources = [source]
            # Source Nodes 갱신
            all_nodes_df.at[target_idx[0], "Source Nodes"] = current_sources

# Source Nodes를 문자열로 변환 (콤마로 연결)
all_nodes_df["Source Nodes"] = all_nodes_df["Source Nodes"].apply(
    lambda x: ", ".join(x) if isinstance(x, list) else ""
)

# 정렬: 중심 노드 기준, 빈도수 기준 내림차순
all_nodes_df = all_nodes_df.sort_values(by=["Node Type", "Frequency"], ascending=[True, False])

# 결과 저장
output_file = "network_nodes_data_dry_positive_negative.xlsx"
all_nodes_df.to_excel(output_file, index=False)

print(f"Processed node data saved to '{output_file}'")

[DEBUG] 모든 노드 데이터:
    Node ID Node Type  Label Sentiment  Frequency Source Nodes
0  만족합니다_긍정     연관 노드  만족합니다        긍정          1           []
1       건성은     중심 노드    건성은       무감정         10           []
2    피부가_긍정     연관 노드    피부가        긍정          7           []
3     악건성이라     중심 노드  악건성이라       무감정          9           []
4   피부라서_긍정     연관 노드   피부라서        긍정          3           []
5        건성     중심 노드     건성       무감정        220           []
6     워낙_긍정     연관 노드     워낙        긍정          3           []
7      건성이라     중심 노드   건성이라       무감정         81           []
8   어머니는_긍정     연관 노드   어머니는        긍정          2           []
9    특성상_긍정     연관 노드    특성상        긍정          1           []
[DEBUG] 모든 엣지 데이터:
Source: 건성은, Target: 만족합니다_긍정
Source: 악건성이라, Target: 피부가_긍정
Source: 건성, Target: 피부라서_긍정
Source: 건성이라, Target: 워낙_긍정
Source: 건성이라, Target: 어머니는_긍정
Source: 건성에, Target: 특성상_긍정
Source: 건성에게, Target: 단비같은_긍정
Source: 건성인, Target: 지인들이_긍정
Source: 악건성, Target: 피부의_긍정
Source: 

## 지성

In [94]:
from pyvis.network import Network
import pandas as pd
import re
from collections import Counter
from konlpy.tag import Okt

# 형태소 분석기 초기화
okt = Okt()

# 불용어 리스트 정의 (한국어 불용어)
with open(r"\Users\kyn03\OneDrive\바탕 화면\project_file\stopwords-ko(한국어불용어).txt", encoding='utf-8') as f:
    stopwords = f.read().splitlines()

# 추가 불용어 정의
stopwords += ['좀', '것', '달바', '미스트', '스프레이', '1으로', '1이라', '1에', '너무', '느낌', '잘', '그냥', '많이', 
              '있어서', '같아요', '스프레이가', '미스트가', '미스트는', '약간의', '생각보다', '있으며', '살짝', '같아서', 
              '느낌이', '저는', '진짜', '엄청', '합니다', '더', '안', '느낌은', '효과는', '조금', '분들은', '크림미스트라서',
              '크림미스트라', '크림', '크림이', '미스트라', '미스트라서', '그런지', '못', '이거', '완전', '아벤느', '다', '개', '걸',
              '매우','음','확','전','제','달바는','막','넘','너무너무','특히','근데','취향이','정말','덜','뭔가',
              '저의','굉장히','아니지만','아니고','해야할까요','제품','시',
              '있는데','샀는데','아니라','기분','썼을것같은데','바꿔봤는데','그런','사봤는데','해야할까요','저에게는',
              '하는데','아주','알겠더라구용','되게','나요','나서','나면서','진심','나니까','저한텐','저한테는','구매했어요',]

# 데이터 로드
data = pd.read_excel(r"\Users\kyn03\OneDrive\바탕 화면\project_file\sorted_merged_dalba.xlsx")
data = data.where(pd.notna(data), None)  # 결측치 처리
data = data.reset_index()

# 인덱스 넘버가 3 + 7 * n인 경우 필터링하여 'Aspect2 Mapped Labels' 행 필터링
filtered_data = data.iloc[[i for i in range(len(data)) if (i - 2) % 7 == 0]]

# 키워드 설정
aspect_keywords = ["지성"]

# 텍스트 전처리 함수
def preprocess_text(text):
    """텍스트에서 특수문자 제거 및 불용어 필터링"""
    cleaned_text = re.sub(r"[^가-힣0-9\s]", "", text)
    tokens = [word for word in cleaned_text.split() if word not in stopwords]
    return tokens

# 네트워크 그래프 생성을 위한 데이터 수집
nodes_data = set()  # 중복 방지를 위해 set 사용
all_previous_tokens = []

# 키워드와 관련된 단어 연결
for idx, row in filtered_data.iterrows():
    original_idx = data[data['index'] == row['index']].index[0]
    tokenized_text_index = original_idx + 3  # 'Tokenized Text' 행
    sentiment_row_index = tokenized_text_index - 5  # 감정 데이터는 동일 열의 -5행에 위치

    if tokenized_text_index < len(data):

        for col_index in range(2, len(data.columns)):  # 각 열을 순회
            tokenized_text = str(data.iat[tokenized_text_index, col_index])

            # 감정 데이터 가져오기
            try:
                sentiment_value = str(data.iat[sentiment_row_index, col_index]).strip()
            except Exception as e:
                print(f"[ERROR] 감정 데이터를 가져오는 중 에러 발생: {e}")
                sentiment_value = None

            # Null 또는 빈 문자열 확인
            if not tokenized_text or tokenized_text.strip() == "":
                continue

            # 키워드 포함 여부 확인
            if not any(keyword in tokenized_text for keyword in aspect_keywords):
                continue

            # 키워드가 포함된 경우 처리
            print(f"[INFO] 키워드 포함 텍스트: {tokenized_text}, 감정: {sentiment_value}")

            # 기준 셀의 왼쪽과 오른쪽 셀 값을 추출
            left_text = (
                str(data.iat[tokenized_text_index, col_index - 1])
                if col_index > 2  # 왼쪽 셀이 존재하는 경우
                else None
            )
            right_text = (
                str(data.iat[tokenized_text_index, col_index + 1])
                if col_index < len(data.columns) - 1  # 오른쪽 셀이 존재하는 경우
                else None
            )

            # 제외할 접미사를 리스트로 관리
            EXCLUDED_SUFFIXES = ("-")
            EXCLUDED_KEYWORDS = ["-"]
            # 왼쪽 셀 추가
            if left_text and left_text.strip():
                stripped_left_text = left_text.strip()  # 공백 제거
                # 접미사 검사
                if any(stripped_left_text.endswith(suffix) for suffix in EXCLUDED_SUFFIXES):
                    print(f"[INFO] 왼쪽 셀이 '{EXCLUDED_SUFFIXES}' 중 하나로 끝나서 제외됨: {stripped_left_text}")

                elif any(keyword in stripped_left_text for keyword in EXCLUDED_KEYWORDS):
                    print(f"[INFO] 왼쪽 셀이 제외 키워드를 포함하여 제외됨: {stripped_left_text}")
                else:
                    nodes_data.add((tokenized_text, stripped_left_text, tokenized_text_index, col_index))  # set에 추가
                    all_previous_tokens.append(stripped_left_text)

            # 오른쪽 셀 추가
            if right_text and right_text.strip():
                stripped_right_text = right_text.strip()  # 공백 제거
                # 키워드 포함 여부 확인
                if any(keyword in stripped_right_text for keyword in EXCLUDED_KEYWORDS):
                    print(f"[INFO] 오른쪽 셀이 제외 키워드를 포함하여 제외됨: {stripped_right_text}")
                else:
                    nodes_data.add((tokenized_text, right_text.strip(),tokenized_text_index,col_index))  # set에 추가
                    all_previous_tokens.append(right_text.strip())

 

# 빈도수 카운터 생성
if not all_previous_tokens:
    print("[ERROR] 관련된 서술어가 없습니다. 전처리 또는 데이터 로드 로직을 확인하세요.")
else:
    # 감정별로 빈도수를 따로 저장하기 위한 딕셔너리
    sentiment_frequency = {
        "긍정": Counter(),
        "부정": Counter(),
        "중립": Counter()
    }

    # 전체 빈도수 저장
    total_frequency = Counter()

    # 빈도수 계산
    for tokenized_text, related_text, tokenized_text_index, col_index in nodes_data:
        # 감정 값 가져오기
        try:
            sentiment_value = str(data.iat[tokenized_text_index - 5, col_index]).strip()
            # B-긍정, I-긍정 같은 값을 "긍정"으로 통합
            if sentiment_value.startswith("B-") or sentiment_value.startswith("I-"):
                sentiment_value = sentiment_value[2:]
        except Exception as e:
            print(f"[ERROR] 감정 데이터를 가져오는 중 에러 발생: {e}")
            sentiment_value = None

        # 유효한 감정 값인지 확인
        if sentiment_value in sentiment_frequency:
            sentiment_frequency[sentiment_value][related_text] += 1
            total_frequency[related_text] += 1  # 전체 빈도수 업데이트




[INFO] 키워드 포함 텍스트: 지성, 감정: O
[INFO] 키워드 포함 텍스트: 지성, 감정: B-부정
[INFO] 키워드 포함 텍스트: 지성, 감정: O
[INFO] 키워드 포함 텍스트: 지성, 감정: B-부정
[INFO] 키워드 포함 텍스트: 지성, 감정: O
[INFO] 키워드 포함 텍스트: 지성, 감정: B-부정
[INFO] 키워드 포함 텍스트: 지성, 감정: B-부정
[INFO] 키워드 포함 텍스트: 지성, 감정: B-부정
[INFO] 키워드 포함 텍스트: 지성, 감정: B-부정
[INFO] 키워드 포함 텍스트: 지성, 감정: B-부정
[INFO] 키워드 포함 텍스트: 지성, 감정: B-부정
[INFO] 키워드 포함 텍스트: 지성, 감정: B-부정
[INFO] 키워드 포함 텍스트: 지성, 감정: B-부정
[INFO] 키워드 포함 텍스트: 지성, 감정: B-부정
[INFO] 키워드 포함 텍스트: 지성에도, 감정: B-긍정
[INFO] 키워드 포함 텍스트: 지성피부에도, 감정: B-긍정
[INFO] 키워드 포함 텍스트: 지성분들도, 감정: I-긍정
[INFO] 키워드 포함 텍스트: 지성, 감정: O
[INFO] 키워드 포함 텍스트: 지성에, 감정: O
[INFO] 키워드 포함 텍스트: 지성인데도, 감정: B-부정
[INFO] 키워드 포함 텍스트: 지성이고, 감정: I-긍정
[INFO] 키워드 포함 텍스트: 지성이든, 감정: I-긍정
[INFO] 키워드 포함 텍스트: 지성분들, 감정: B-긍정
[INFO] 키워드 포함 텍스트: 지성이나, 감정: B-긍정
[INFO] 키워드 포함 텍스트: 지성분들은, 감정: I-부정
[INFO] 키워드 포함 텍스트: 지성인데도, 감정: B-긍정
[INFO] 키워드 포함 텍스트: 지성, 감정: O
[INFO] 키워드 포함 텍스트: 지성들, 감정: O
[INFO] 키워드 포함 텍스트: 지성, 감정: O
[INFO] 키워드 포함 텍스트: 지성한텐, 감정: B-부정
[INFO] 키워드 포함 텍스트: 지성인분들은, 감정: I-긍

In [95]:
# 네트워크 그래프 생성
net = Network(notebook=True, cdn_resources="in_line")
net.toggle_physics(True)

# 중심 노드와 연관 노드 간 연결 정보 기록
connected_nodes = {}
center_frequency = Counter()

# 그래프 노드 및 엣지 추가
EXCLUDED_MAIN_KEYWORDS = ["-"]

for main_token, related_token, sentiment_row_index, col_index in nodes_data:
    main_node_key = main_token.strip()  # 중심 노드 키 생성
    print(f"[DEBUG] Processing pair: main_token={main_token}, related_token={related_token}")

    # 유효성 검증 및 불용어 필터링
    if not re.match(r"^[가-힣]+$", main_node_key) or not re.match(r"^[가-힣]+$", related_token):
        print(f"[INFO] main_token 또는 related_token이 유효하지 않음: {main_token}, {related_token}")
        continue
    if main_token in stopwords or related_token in stopwords:
        print(f"[INFO] 불용어로 제외됨: {main_token} 또는 {related_token}")
        continue
    if any(keyword in main_node_key for keyword in EXCLUDED_MAIN_KEYWORDS):
        print(f"[INFO] 중심 노드가 제외 키워드를 포함하여 추가되지 않음: {main_node_key}")
        continue

    # 감정 값 가져오기
    related_token_sentiment = None
    try:
        related_token_sentiment = str(data.iat[sentiment_row_index - 5, col_index]).strip()
        if related_token_sentiment.startswith("B-") or related_token_sentiment.startswith("I-"):
            related_token_sentiment = related_token_sentiment[2:]  # 통합
    except Exception as e:
        print(f"[ERROR] related_token 감정 데이터 가져오기 실패: {e}")
        continue

    # 연관 노드 추가
    related_token_frequency = sentiment_frequency.get(related_token_sentiment, {}).get(related_token, 0)
    if related_token_frequency >= 1:
        if "중립" in related_token_sentiment or "O" in related_token_sentiment:
            print(f"[INFO] 연관 노드가 '중립' 또는 'O' 감정으로 추가되지 않음: {related_token}")
            continue

        related_node_key = f"{related_token}_{related_token_sentiment}"
        if related_node_key not in net.get_nodes():
            node_size = min(15 + related_token_frequency * 2, 50)
            related_token_color = {
                "긍정": "#6EC664",
                "중립": "#BDC3C7",
                "부정": "#E74C3C"
            }.get(related_token_sentiment, "black")
            net.add_node(
                related_node_key,
                label=related_token,
                color=related_token_color,
                size=node_size,
                title=f"{related_token} (빈도: {related_token_frequency})"
            )
            print(f"[DEBUG] 추가된 연관 노드: {related_node_key}")

        # 엣지 및 중심 노드 추가
        if main_node_key not in net.get_nodes():
            net.add_node(
                main_node_key,
                label=main_token,
                color={"background": "white", "border": "black"},
                size=25,
                title=f"{main_token} (빈도: 0)"  # 빈도수는 추후 갱신
            )
        # 엣지 추가
        net.add_edge(main_node_key, related_node_key, color=related_token_color)
        print(f"[DEBUG] 엣지 추가됨: {main_node_key} -> {related_node_key}")
        connected_nodes.setdefault(main_node_key, []).append(related_node_key)

# 중심 노드 빈도 계산 (화면에 표시된 연관 노드 기반)
center_frequency.clear()  # 기존 빈도수 초기화
for main_node_key, related_nodes in connected_nodes.items():
    for related_node_key in related_nodes:
        related_token = related_node_key.split("_")[0]
        related_token_sentiment = related_node_key.split("_")[1]
        related_token_frequency = sentiment_frequency.get(related_token_sentiment, {}).get(related_token, 0)
        center_frequency[main_node_key] += related_token_frequency
        print(f"[DEBUG] 중심 노드 '{main_node_key}'에 '{related_token}'의 빈도 {related_token_frequency} 추가됨")

# 중심 노드 빈도 업데이트 (화면에 표시된 값으로 갱신)
for node in net.nodes:
    if node["id"] in center_frequency:
        node["title"] = f"{node['id']} (빈도: {center_frequency[node['id']]})"
        print(f"[DEBUG] 중심 노드 '{node['id']}'의 빈도 갱신됨: {center_frequency[node['id']]}")

# 디버깅: 최종 중심 노드 빈도수 출력
print("[DEBUG] 최종 중심 노드 빈도수:")
for node, freq in center_frequency.items():
    print(f"{node}: {freq}")

# 네트워크 그래프 시각화
net.show("greasy_positive_negative.html")


[DEBUG] Processing pair: main_token=지성, related_token=그러나
[INFO] 불용어로 제외됨: 지성 또는 그러나
[DEBUG] Processing pair: main_token=지성, related_token=민감성
[DEBUG] 추가된 연관 노드: 민감성_부정
[DEBUG] 엣지 추가됨: 지성 -> 민감성_부정
[DEBUG] Processing pair: main_token=지성피부인, related_token=분들이
[DEBUG] 추가된 연관 노드: 분들이_긍정
[DEBUG] 엣지 추가됨: 지성피부인 -> 분들이_긍정
[DEBUG] Processing pair: main_token=파워지성도마찬가지임, related_token=채워야하는건
[DEBUG] Processing pair: main_token=지성이, related_token=쓰기에는
[DEBUG] 추가된 연관 노드: 쓰기에는_부정
[DEBUG] 엣지 추가됨: 지성이 -> 쓰기에는_부정
[DEBUG] Processing pair: main_token=지성인, related_token=복합성
[DEBUG] 추가된 연관 노드: 복합성_긍정
[DEBUG] 엣지 추가됨: 지성인 -> 복합성_긍정
[DEBUG] Processing pair: main_token=수분부족지성이라, related_token=유분기가
[DEBUG] Processing pair: main_token=지성, related_token=,
[INFO] main_token 또는 related_token이 유효하지 않음: 지성, ,
[DEBUG] Processing pair: main_token=지성, related_token=!
[INFO] main_token 또는 related_token이 유효하지 않음: 지성, !
[DEBUG] Processing pair: main_token=지성이라서, related_token=👍
[INFO] main_token 또는 related_token이 유효하지 않음

In [96]:
import pandas as pd
import re

# 네트워크에 표시된 모든 노드 데이터 가져오기
all_nodes_data = []
for node in net.nodes:
    # 노드 정보 추출
    node_id = node["id"]
    node_label = node.get("label", "")
    node_color = node.get("color", "")
    node_title = node.get("title", "")  # 빈도수 정보 포함 가능

    # 노드 종류 결정
    if "_" in node_id:  # "_"이 포함된 경우 연관 노드
        node_type = "연관 노드"
    else:  # "_"이 없는 경우 중심 노드
        node_type = "중심 노드"

    # 감정 추출 (색상 기반)
    if node_color == "#6EC664":  # 연한 녹색
        node_sentiment = "긍정"
    elif node_color == "#BDC3C7":  # 연한 회색 (중립)
        node_sentiment = "중립"
    elif node_color == "#E74C3C":  # 연한 빨간색 (부정)
        node_sentiment = "부정"
    else:
        node_sentiment = "무감정"

    # 빈도수 추출 (Title에서 추출)
    try:
        node_frequency = int(re.search(r"\(빈도: (\d+)\)", node_title).group(1))
    except AttributeError:
        node_frequency = 0  # 빈도수 정보가 없으면 0으로 설정

    # 데이터 추가
    all_nodes_data.append({
        "Node ID": node_id,
        "Node Type": node_type,
        "Label": node_label,
        "Sentiment": node_sentiment,
        "Frequency": node_frequency,
        "Source Nodes": []  # 초기값으로 빈 리스트 설정 (연관 노드일 경우 추가적으로 설정)
    })

# DataFrame으로 변환
all_nodes_df = pd.DataFrame(all_nodes_data)

# 디버깅: 모든 노드 출력
print("[DEBUG] 모든 노드 데이터:")
print(all_nodes_df.head(10))

# 디버깅: 모든 엣지 데이터 출력
print("[DEBUG] 모든 엣지 데이터:")
for edge in net.edges:
    print(f"Source: {edge['from']}, Target: {edge['to']}")

# 엣지 데이터를 활용하여 Source Nodes 설정
for edge in net.edges:
    source = edge["from"]
    target = edge["to"]

    # 디버깅: 현재 처리 중인 엣지
    print(f"[DEBUG] 처리 중인 엣지: Source={source}, Target={target}")

    # 타겟 노드가 연관 노드인 경우 Source Nodes에 소스 노드 추가
    if "_" in target:  # 타겟 노드가 연관 노드
        target_idx = all_nodes_df[all_nodes_df["Node ID"] == target].index
        if not target_idx.empty:
            current_sources = all_nodes_df.at[target_idx[0], "Source Nodes"]
            if isinstance(current_sources, list):  # 리스트 형태인 경우
                current_sources.append(source)
            else:  # 초기값이 없거나 비어 있을 경우
                current_sources = [source]
            # Source Nodes 갱신
            all_nodes_df.at[target_idx[0], "Source Nodes"] = current_sources

# Source Nodes를 문자열로 변환 (콤마로 연결)
all_nodes_df["Source Nodes"] = all_nodes_df["Source Nodes"].apply(
    lambda x: ", ".join(x) if isinstance(x, list) else ""
)

# 정렬: 중심 노드 기준, 빈도수 기준 내림차순
all_nodes_df = all_nodes_df.sort_values(by=["Node Type", "Frequency"], ascending=[True, False])

# 결과 저장
output_file = "network_nodes_data_greasy_positive_negative.xlsx"
all_nodes_df.to_excel(output_file, index=False)

print(f"Processed node data saved to '{output_file}'")

[DEBUG] 모든 노드 데이터:
   Node ID Node Type  Label Sentiment  Frequency Source Nodes
0   민감성_부정     연관 노드    민감성        부정          2           []
1       지성     중심 노드     지성       무감정        141           []
2   분들이_긍정     연관 노드    분들이        긍정          2           []
3    지성피부인     중심 노드  지성피부인       무감정          5           []
4  쓰기에는_부정     연관 노드   쓰기에는        부정          1           []
5      지성이     중심 노드    지성이       무감정          6           []
6   복합성_긍정     연관 노드    복합성        긍정          5           []
7      지성인     중심 노드    지성인       무감정         39           []
8  남자분은_긍정     연관 노드   남자분은        긍정          1           []
9   피부가_긍정     연관 노드    피부가        긍정          2           []
[DEBUG] 모든 엣지 데이터:
Source: 지성, Target: 민감성_부정
Source: 지성피부인, Target: 분들이_긍정
Source: 지성이, Target: 쓰기에는_부정
Source: 지성인, Target: 복합성_긍정
Source: 지성인, Target: 남자분은_긍정
Source: 지성인데도, Target: 피부가_긍정
Source: 지성, Target: 복합성_긍정
Source: 지성인데, Target: 수부지_긍정
Source: 지성, Target: 피부에게는_부정
Source: 지성, Target: 복합

## 복합성

In [97]:
from pyvis.network import Network
import pandas as pd
import re
from collections import Counter
from konlpy.tag import Okt

# 형태소 분석기 초기화
okt = Okt()

# 불용어 리스트 정의 (한국어 불용어)
with open(r"\Users\kyn03\OneDrive\바탕 화면\project_file\stopwords-ko(한국어불용어).txt", encoding='utf-8') as f:
    stopwords = f.read().splitlines()

# 추가 불용어 정의
stopwords += ['좀', '것', '달바', '미스트', '스프레이', '1으로', '1이라', '1에', '너무', '느낌', '잘', '그냥', '많이', 
              '있어서', '같아요', '스프레이가', '미스트가', '미스트는', '약간의', '생각보다', '있으며', '살짝', '같아서', 
              '느낌이', '저는', '진짜', '엄청', '합니다', '더', '안', '느낌은', '효과는', '조금', '분들은', '크림미스트라서',
              '크림미스트라', '크림', '크림이', '미스트라', '미스트라서', '그런지', '못', '이거', '완전', '아벤느', '다', '개', '걸',
              '매우','음','확','전','제','달바는','막','넘','너무너무','특히','근데','취향이','정말','덜','뭔가',
              '저의','굉장히','아니지만','아니고','해야할까요','제품','시',
              '있는데','샀는데','아니라','기분','썼을것같은데','바꿔봤는데','그런','사봤는데','해야할까요','저에게는',
              '하는데','아주','알겠더라구용','되게','나요','나서','나면서','진심','나니까','저한텐','저한테는','구매했어요',]

# 데이터 로드
data = pd.read_excel(r"\Users\kyn03\OneDrive\바탕 화면\project_file\sorted_merged_dalba.xlsx")
data = data.where(pd.notna(data), None)  # 결측치 처리
data = data.reset_index()

# 인덱스 넘버가 3 + 7 * n인 경우 필터링하여 'Aspect2 Mapped Labels' 행 필터링
filtered_data = data.iloc[[i for i in range(len(data)) if (i - 2) % 7 == 0]]

# 키워드 설정
aspect_keywords = ["복합성"]

# 텍스트 전처리 함수
def preprocess_text(text):
    """텍스트에서 특수문자 제거 및 불용어 필터링"""
    cleaned_text = re.sub(r"[^가-힣0-9\s]", "", text)
    tokens = [word for word in cleaned_text.split() if word not in stopwords]
    return tokens

# 네트워크 그래프 생성을 위한 데이터 수집
nodes_data = set()  # 중복 방지를 위해 set 사용
all_previous_tokens = []

# 키워드와 관련된 단어 연결
for idx, row in filtered_data.iterrows():
    original_idx = data[data['index'] == row['index']].index[0]
    tokenized_text_index = original_idx + 3  # 'Tokenized Text' 행
    sentiment_row_index = tokenized_text_index - 5  # 감정 데이터는 동일 열의 -5행에 위치

    if tokenized_text_index < len(data):

        for col_index in range(2, len(data.columns)):  # 각 열을 순회
            tokenized_text = str(data.iat[tokenized_text_index, col_index])

            # 감정 데이터 가져오기
            try:
                sentiment_value = str(data.iat[sentiment_row_index, col_index]).strip()
            except Exception as e:
                print(f"[ERROR] 감정 데이터를 가져오는 중 에러 발생: {e}")
                sentiment_value = None

            # Null 또는 빈 문자열 확인
            if not tokenized_text or tokenized_text.strip() == "":
                continue

            # 키워드 포함 여부 확인
            if not any(keyword in tokenized_text for keyword in aspect_keywords):
                continue

            # 키워드가 포함된 경우 처리
            print(f"[INFO] 키워드 포함 텍스트: {tokenized_text}, 감정: {sentiment_value}")

            # 기준 셀의 왼쪽과 오른쪽 셀 값을 추출
            left_text = (
                str(data.iat[tokenized_text_index, col_index - 1])
                if col_index > 2  # 왼쪽 셀이 존재하는 경우
                else None
            )
            right_text = (
                str(data.iat[tokenized_text_index, col_index + 1])
                if col_index < len(data.columns) - 1  # 오른쪽 셀이 존재하는 경우
                else None
            )

            # 제외할 접미사를 리스트로 관리
            EXCLUDED_SUFFIXES = ("-")
            EXCLUDED_KEYWORDS = ["-"]
            # 왼쪽 셀 추가
            if left_text and left_text.strip():
                stripped_left_text = left_text.strip()  # 공백 제거
                # 접미사 검사
                if any(stripped_left_text.endswith(suffix) for suffix in EXCLUDED_SUFFIXES):
                    print(f"[INFO] 왼쪽 셀이 '{EXCLUDED_SUFFIXES}' 중 하나로 끝나서 제외됨: {stripped_left_text}")

                elif any(keyword in stripped_left_text for keyword in EXCLUDED_KEYWORDS):
                    print(f"[INFO] 왼쪽 셀이 제외 키워드를 포함하여 제외됨: {stripped_left_text}")
                else:
                    nodes_data.add((tokenized_text, stripped_left_text, tokenized_text_index, col_index))  # set에 추가
                    all_previous_tokens.append(stripped_left_text)

            # 오른쪽 셀 추가
            if right_text and right_text.strip():
                stripped_right_text = right_text.strip()  # 공백 제거
                # 키워드 포함 여부 확인
                if any(keyword in stripped_right_text for keyword in EXCLUDED_KEYWORDS):
                    print(f"[INFO] 오른쪽 셀이 제외 키워드를 포함하여 제외됨: {stripped_right_text}")
                else:
                    nodes_data.add((tokenized_text, right_text.strip(),tokenized_text_index,col_index))  # set에 추가
                    all_previous_tokens.append(right_text.strip())

 

# 빈도수 카운터 생성
if not all_previous_tokens:
    print("[ERROR] 관련된 서술어가 없습니다. 전처리 또는 데이터 로드 로직을 확인하세요.")
else:
    # 감정별로 빈도수를 따로 저장하기 위한 딕셔너리
    sentiment_frequency = {
        "긍정": Counter(),
        "부정": Counter(),
        "중립": Counter()
    }

    # 전체 빈도수 저장
    total_frequency = Counter()

    # 빈도수 계산
    for tokenized_text, related_text, tokenized_text_index, col_index in nodes_data:
        # 감정 값 가져오기
        try:
            sentiment_value = str(data.iat[tokenized_text_index - 5, col_index]).strip()
            # B-긍정, I-긍정 같은 값을 "긍정"으로 통합
            if sentiment_value.startswith("B-") or sentiment_value.startswith("I-"):
                sentiment_value = sentiment_value[2:]
        except Exception as e:
            print(f"[ERROR] 감정 데이터를 가져오는 중 에러 발생: {e}")
            sentiment_value = None

        # 유효한 감정 값인지 확인
        if sentiment_value in sentiment_frequency:
            sentiment_frequency[sentiment_value][related_text] += 1
            total_frequency[related_text] += 1  # 전체 빈도수 업데이트



[INFO] 키워드 포함 텍스트: 복합성에, 감정: I-긍정
[INFO] 키워드 포함 텍스트: 복합성인, 감정: B-긍정
[INFO] 키워드 포함 텍스트: 복합성이라, 감정: O
[INFO] 키워드 포함 텍스트: 복합성이라, 감정: O
[INFO] 키워드 포함 텍스트: 복합성이지만, 감정: B-긍정
[INFO] 키워드 포함 텍스트: 복합성, 감정: O
[INFO] 키워드 포함 텍스트: 복합성인, 감정: B-긍정
[INFO] 키워드 포함 텍스트: 복합성, 감정: B-긍정
[INFO] 키워드 포함 텍스트: 복합성인데, 감정: O
[INFO] 키워드 포함 텍스트: 복합성, 감정: B-긍정
[INFO] 키워드 포함 텍스트: 지복합성, 감정: O
[INFO] 키워드 포함 텍스트: 복합성, 감정: O
[INFO] 키워드 포함 텍스트: 복합성, 감정: O
[INFO] 키워드 포함 텍스트: 복합성피부, 감정: O
[INFO] 키워드 포함 텍스트: 복합성피부, 감정: O
[INFO] 키워드 포함 텍스트: 복합성보단, 감정: I-긍정
[INFO] 키워드 포함 텍스트: 복합성피부, 감정: I-긍정
[INFO] 키워드 포함 텍스트: 복합성피부, 감정: I-긍정
[INFO] 키워드 포함 텍스트: 복합성피부, 감정: I-긍정
[INFO] 키워드 포함 텍스트: 복합성피부, 감정: I-긍정
[INFO] 키워드 포함 텍스트: 복합성, 감정: B-긍정
[INFO] 키워드 포함 텍스트: 복합성, 감정: O
[INFO] 키워드 포함 텍스트: 복합성이나, 감정: B-부정
[INFO] 키워드 포함 텍스트: 복합성이, 감정: I-긍정
[INFO] 키워드 포함 텍스트: 복합성, 감정: B-긍정
[INFO] 키워드 포함 텍스트: 복합성, 감정: B-부정
[INFO] 키워드 포함 텍스트: 복합성, 감정: B-긍정
[INFO] 키워드 포함 텍스트: 복합성인데, 감정: O
[INFO] 키워드 포함 텍스트: 복합성, 감정: B-긍정
[INFO] 키워드 포함 텍스트: 복합성, 감정: B-긍정
[INFO] 키워드 

In [98]:
# 네트워크 그래프 생성
net = Network(notebook=True, cdn_resources="in_line")
net.toggle_physics(True)

# 중심 노드와 연관 노드 간 연결 정보 기록
connected_nodes = {}
center_frequency = Counter()

# 그래프 노드 및 엣지 추가
EXCLUDED_MAIN_KEYWORDS = ["-"]

for main_token, related_token, sentiment_row_index, col_index in nodes_data:
    main_node_key = main_token.strip()  # 중심 노드 키 생성
    print(f"[DEBUG] Processing pair: main_token={main_token}, related_token={related_token}")

    # 유효성 검증 및 불용어 필터링
    if not re.match(r"^[가-힣]+$", main_node_key) or not re.match(r"^[가-힣]+$", related_token):
        print(f"[INFO] main_token 또는 related_token이 유효하지 않음: {main_token}, {related_token}")
        continue
    if main_token in stopwords or related_token in stopwords:
        print(f"[INFO] 불용어로 제외됨: {main_token} 또는 {related_token}")
        continue
    if any(keyword in main_node_key for keyword in EXCLUDED_MAIN_KEYWORDS):
        print(f"[INFO] 중심 노드가 제외 키워드를 포함하여 추가되지 않음: {main_node_key}")
        continue

    # 감정 값 가져오기
    related_token_sentiment = None
    try:
        related_token_sentiment = str(data.iat[sentiment_row_index - 5, col_index]).strip()
        if related_token_sentiment.startswith("B-") or related_token_sentiment.startswith("I-"):
            related_token_sentiment = related_token_sentiment[2:]  # 통합
    except Exception as e:
        print(f"[ERROR] related_token 감정 데이터 가져오기 실패: {e}")
        continue

    # 연관 노드 추가
    related_token_frequency = sentiment_frequency.get(related_token_sentiment, {}).get(related_token, 0)
    if related_token_frequency >= 1:
        if "중립" in related_token_sentiment or "O" in related_token_sentiment:
            print(f"[INFO] 연관 노드가 '중립' 또는 'O' 감정으로 추가되지 않음: {related_token}")
            continue

        related_node_key = f"{related_token}_{related_token_sentiment}"
        if related_node_key not in net.get_nodes():
            node_size = min(15 + related_token_frequency * 2, 50)
            related_token_color = {
                "긍정": "#6EC664",
                "중립": "#BDC3C7",
                "부정": "#E74C3C"
            }.get(related_token_sentiment, "black")
            net.add_node(
                related_node_key,
                label=related_token,
                color=related_token_color,
                size=node_size,
                title=f"{related_token} (빈도: {related_token_frequency})"
            )
            print(f"[DEBUG] 추가된 연관 노드: {related_node_key}")

        # 엣지 및 중심 노드 추가
        if main_node_key not in net.get_nodes():
            net.add_node(
                main_node_key,
                label=main_token,
                color={"background": "white", "border": "black"},
                size=25,
                title=f"{main_token} (빈도: 0)"  # 빈도수는 추후 갱신
            )
        # 엣지 추가
        net.add_edge(main_node_key, related_node_key, color=related_token_color)
        print(f"[DEBUG] 엣지 추가됨: {main_node_key} -> {related_node_key}")
        connected_nodes.setdefault(main_node_key, []).append(related_node_key)

# 중심 노드 빈도 계산 (화면에 표시된 연관 노드 기반)
center_frequency.clear()  # 기존 빈도수 초기화
for main_node_key, related_nodes in connected_nodes.items():
    for related_node_key in related_nodes:
        related_token = related_node_key.split("_")[0]
        related_token_sentiment = related_node_key.split("_")[1]
        related_token_frequency = sentiment_frequency.get(related_token_sentiment, {}).get(related_token, 0)
        center_frequency[main_node_key] += related_token_frequency
        print(f"[DEBUG] 중심 노드 '{main_node_key}'에 '{related_token}'의 빈도 {related_token_frequency} 추가됨")

# 중심 노드 빈도 업데이트 (화면에 표시된 값으로 갱신)
for node in net.nodes:
    if node["id"] in center_frequency:
        node["title"] = f"{node['id']} (빈도: {center_frequency[node['id']]})"
        print(f"[DEBUG] 중심 노드 '{node['id']}'의 빈도 갱신됨: {center_frequency[node['id']]}")

# 디버깅: 최종 중심 노드 빈도수 출력
print("[DEBUG] 최종 중심 노드 빈도수:")
for node, freq in center_frequency.items():
    print(f"{node}: {freq}")

# 네트워크 그래프 시각화
net.show("complex_positive_negative.html")


[DEBUG] Processing pair: main_token=복합성, related_token=지성
[DEBUG] 추가된 연관 노드: 지성_긍정
[DEBUG] 엣지 추가됨: 복합성 -> 지성_긍정
[DEBUG] Processing pair: main_token=복합성이라, related_token=아무래도
[DEBUG] 추가된 연관 노드: 아무래도_긍정
[DEBUG] 엣지 추가됨: 복합성이라 -> 아무래도_긍정
[DEBUG] Processing pair: main_token=복합성도, related_token=,
[INFO] main_token 또는 related_token이 유효하지 않음: 복합성도, ,
[DEBUG] Processing pair: main_token=복합성보단, related_token=건성
[DEBUG] 추가된 연관 노드: 건성_긍정
[DEBUG] 엣지 추가됨: 복합성보단 -> 건성_긍정
[DEBUG] Processing pair: main_token=지복합성, related_token=피부가
[DEBUG] 추가된 연관 노드: 피부가_긍정
[DEBUG] 엣지 추가됨: 지복합성 -> 피부가_긍정
[DEBUG] Processing pair: main_token=복합성인데, related_token=수부지
[DEBUG] Processing pair: main_token=복합성, related_token=피부인
[DEBUG] 추가된 연관 노드: 피부인_긍정
[DEBUG] 엣지 추가됨: 복합성 -> 피부인_긍정
[DEBUG] Processing pair: main_token=복합성이구, related_token=피부타입은
[DEBUG] Processing pair: main_token=복합성, related_token=,
[INFO] main_token 또는 related_token이 유효하지 않음: 복합성, ,
[DEBUG] Processing pair: main_token=복합성, related_token=저는
[INFO] 불용어로 제외됨:

In [99]:
import pandas as pd
import re

# 네트워크에 표시된 모든 노드 데이터 가져오기
all_nodes_data = []
for node in net.nodes:
    # 노드 정보 추출
    node_id = node["id"]
    node_label = node.get("label", "")
    node_color = node.get("color", "")
    node_title = node.get("title", "")  # 빈도수 정보 포함 가능

    # 노드 종류 결정
    if "_" in node_id:  # "_"이 포함된 경우 연관 노드
        node_type = "연관 노드"
    else:  # "_"이 없는 경우 중심 노드
        node_type = "중심 노드"

    # 감정 추출 (색상 기반)
    if node_color == "#6EC664":  # 연한 녹색
        node_sentiment = "긍정"
    elif node_color == "#BDC3C7":  # 연한 회색 (중립)
        node_sentiment = "중립"
    elif node_color == "#E74C3C":  # 연한 빨간색 (부정)
        node_sentiment = "부정"
    else:
        node_sentiment = "무감정"

    # 빈도수 추출 (Title에서 추출)
    try:
        node_frequency = int(re.search(r"\(빈도: (\d+)\)", node_title).group(1))
    except AttributeError:
        node_frequency = 0  # 빈도수 정보가 없으면 0으로 설정

    # 데이터 추가
    all_nodes_data.append({
        "Node ID": node_id,
        "Node Type": node_type,
        "Label": node_label,
        "Sentiment": node_sentiment,
        "Frequency": node_frequency,
        "Source Nodes": []  # 초기값으로 빈 리스트 설정 (연관 노드일 경우 추가적으로 설정)
    })

# DataFrame으로 변환
all_nodes_df = pd.DataFrame(all_nodes_data)

# 디버깅: 모든 노드 출력
print("[DEBUG] 모든 노드 데이터:")
print(all_nodes_df.head(10))

# 디버깅: 모든 엣지 데이터 출력
print("[DEBUG] 모든 엣지 데이터:")
for edge in net.edges:
    print(f"Source: {edge['from']}, Target: {edge['to']}")

# 엣지 데이터를 활용하여 Source Nodes 설정
for edge in net.edges:
    source = edge["from"]
    target = edge["to"]

    # 디버깅: 현재 처리 중인 엣지
    print(f"[DEBUG] 처리 중인 엣지: Source={source}, Target={target}")

    # 타겟 노드가 연관 노드인 경우 Source Nodes에 소스 노드 추가
    if "_" in target:  # 타겟 노드가 연관 노드
        target_idx = all_nodes_df[all_nodes_df["Node ID"] == target].index
        if not target_idx.empty:
            current_sources = all_nodes_df.at[target_idx[0], "Source Nodes"]
            if isinstance(current_sources, list):  # 리스트 형태인 경우
                current_sources.append(source)
            else:  # 초기값이 없거나 비어 있을 경우
                current_sources = [source]
            # Source Nodes 갱신
            all_nodes_df.at[target_idx[0], "Source Nodes"] = current_sources

# Source Nodes를 문자열로 변환 (콤마로 연결)
all_nodes_df["Source Nodes"] = all_nodes_df["Source Nodes"].apply(
    lambda x: ", ".join(x) if isinstance(x, list) else ""
)

# 정렬: 중심 노드 기준, 빈도수 기준 내림차순
all_nodes_df = all_nodes_df.sort_values(by=["Node Type", "Frequency"], ascending=[True, False])

# 결과 저장
output_file = "network_nodes_data_complex_positive_negative.xlsx"
all_nodes_df.to_excel(output_file, index=False)

print(f"Processed node data saved to '{output_file}'")

[DEBUG] 모든 노드 데이터:
   Node ID Node Type  Label Sentiment  Frequency Source Nodes
0    지성_긍정     연관 노드     지성        긍정          2           []
1      복합성     중심 노드    복합성       무감정        109           []
2  아무래도_긍정     연관 노드   아무래도        긍정          1           []
3    복합성이라     중심 노드  복합성이라       무감정          6           []
4    건성_긍정     연관 노드     건성        긍정          4           []
5    복합성보단     중심 노드  복합성보단       무감정          5           []
6   피부가_긍정     연관 노드    피부가        긍정          1           []
7     지복합성     중심 노드   지복합성       무감정          3           []
8   피부인_긍정     연관 노드    피부인        긍정          1           []
9  피부인데_긍정     연관 노드   피부인데        긍정          5           []
[DEBUG] 모든 엣지 데이터:
Source: 복합성, Target: 지성_긍정
Source: 복합성이라, Target: 아무래도_긍정
Source: 복합성보단, Target: 건성_긍정
Source: 지복합성, Target: 피부가_긍정
Source: 복합성, Target: 피부인_긍정
Source: 복합성, Target: 피부인데_긍정
Source: 복합성피부에도, Target: 유분지지_긍정
Source: 지복합성, Target: 산건데_부정
Source: 복합성이라, Target: 적당량_부정
Source: 복합성, Ta